# Matchup Evaluator
This notebook is used to create code and logic that will help analyse an upcoming h2h Yahoo
Fantasy matchup

In [ ]:
import pandas as pd
df = pd.DataFrame([123.4567, 234.5678, 345.6789, 456.7890],
                  index=['foo','bar','baz','quux'],
                  columns=['cost'])
df['foo'] = df['cost']
df['cost'] = df['cost'].map('${:,.2f}'.format)
print(df)

In [ ]:
from src.data import all_functions as af
import pandas as pd

In [ ]:
# Authenticate
sc = af.yahoo_fantasy_api_authentication()
league = af.yahoo_fantasy_league(sc)

In [ ]:
# Nunn 9Cat averages
nunn_of_yall_betta_team = af.team_9cat_average_stats(af.NUNN_OF_YALL_BETTA, league)

In [ ]:
nunn_of_yall_betta_team = nunn_of_yall_betta_team.loc["mean"]
nunn_of_yall_betta_team["PLAYER"] = af.NUNN_OF_YALL_BETTA["name"]
nunn_of_yall_betta_team = pd.DataFrame(nunn_of_yall_betta_team).T
nunn_of_yall_betta_team

In [ ]:
# Crabbe 9Cat averages
magics_johnson_team = af.team_9cat_average_stats(af.MAGICS_JOHNSON, league)

In [ ]:
magics_johnson_team = magics_johnson_team.loc["mean"]
magics_johnson_team["PLAYER"] = af.MAGICS_JOHNSON["name"]
magics_johnson_team = pd.DataFrame(magics_johnson_team).T
magics_johnson_team

In [ ]:
# Drop PLAYER
nunn_of_yall_betta_team.drop("PLAYER", inplace=True, axis=1)
magics_johnson_team.drop("PLAYER", inplace=True, axis=1)

In [ ]:
def columns_to_dtype_float(dataframe):
    """
    Converts dataframe columns to dtype float.
    param dataframe (pandas dataframe): dataframe to change column types
    returns dataframe (pandas dataframe): dataframe with changed column types
    """
    for column in dataframe.columns:
        try:
            dataframe[column] = dataframe[column].astype(float)
        except ValueError:
            pass
    return dataframe

In [ ]:
nunn_of_yall_betta_team = columns_to_dtype_float(nunn_of_yall_betta_team)
magics_johnson_team = columns_to_dtype_float(magics_johnson_team)

In [ ]:
# Difference
matchup_difference = nunn_of_yall_betta_team.sub(magics_johnson_team)
matchup_difference


In [ ]:
matchup_difference.sum(axis=1)

In [ ]:
COLUMN_9CAT_DECIMAL_FORMAT = {af.FIELD_GOAL_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.FREE_THROW_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.THREES_MADE_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.POINTS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.REBOUNDS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.ASSITS_COLUMN : af.TWO_DECIMAL_FORMAT, af.STEALS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.BLOCKS_COLUMN : af.TWO_DECIMAL_FORMAT, af.TURNOVERS_COLUMN : af.TWO_DECIMAL_FORMAT}

def applymap_color_and_format(dataframe):
    """

    """
    fromated_matchup_difference = matchup_difference.style.applymap(
        af.color_negative_red, subset=pd.IndexSlice[:,[af.FIELD_GOAL_PERCENTAGE_COLUMN,
                                                       af.FREE_THROW_PERCENTAGE_COLUMN,
                                                       af.THREES_MADE_COLUMN, af.POINTS_COLUMN,
                                                       af.REBOUNDS_COLUMN, af.ASSITS_COLUMN,
                                                       af.STEALS_COLUMN, af.BLOCKS_COLUMN
                                                       ]]).applymap(
        af.color_negative_red_tov, subset=pd.IndexSlice[:,[af.TURNOVERS_COLUMN]]).format(COLUMN_9CAT_DECIMAL_FORMAT)
    return fromated_matchup_difference

In [ ]:
matchup_difference.style.applymap(
    af.color_negative_red, subset=pd.IndexSlice[:,["FG_PCT", "FT_PCT", "FG3M", "PTS", "REB", "AST",
                                                "STL", "BLK"]]).applymap(
    af.color_negative_red_tov, subset=pd.IndexSlice[:,["TOV"]]).format(COLUMN_9CAT_DECIMAL_FORMAT)

In [ ]:
PLAYER_COLUMN = "PLAYER"
MEAN_ROW = "mean"
NAME_KEY = "name"
COLUMN_9CAT_DECIMAL_FORMAT = {af.FIELD_GOAL_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.FREE_THROW_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.THREES_MADE_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.POINTS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.REBOUNDS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.ASSITS_COLUMN : af.TWO_DECIMAL_FORMAT, af.STEALS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.BLOCKS_COLUMN : af.TWO_DECIMAL_FORMAT, af.TURNOVERS_COLUMN : af.TWO_DECIMAL_FORMAT}

def columns_to_dtype_float(dataframe):
    """
    Converts dataframe columns to dtype float.
    param dataframe (pandas dataframe): dataframe to change column types
    returns dataframe (pandas dataframe): dataframe with changed column types
    """
    for column in dataframe.columns:
        try:
            dataframe[column] = dataframe[column].astype(float)
        except ValueError:
            pass
    return dataframe

def get_and_format_team_9cat(team_dict, league):
    """
    Function first finds the teams 9cat averages by calling the *team_9cat_average_stats*
    function. It then filters to find the team mean and cleans and formats this single row
    dataframe. It changes the dtype of the columns to be floats.
    param team (dictionary): team dictionary
    param league (yahoo oauth object): yahoo authentication object
    returns team_9cat (dataframe): dataframe of team average 9cats
    """
    team_9cat = af.team_9cat_average_stats(team_dict, league)
    team_9cat = team_9cat.loc[MEAN_ROW]
    team_9cat[PLAYER_COLUMN] = team_dict[NAME_KEY]
    team_9cat = pd.DataFrame(team_9cat).T
    team_9cat.drop(PLAYER_COLUMN, inplace=True, axis=1)
    team_9cat = columns_to_dtype_float(team_9cat)
    return team_9cat

def applymap_color_and_format(dataframe):
    """
    Function takes a dataframe containing 9cat stat columns differneces and returns a color coded
    and decimal place formatted dataframe for visual ease.
    param dataframe (pandas dataframe): dataframe to format
    returns dataframe (pandas dataframe): dataframe with formatted and colored values
    """
    colored_matchup_difference = dataframe.style.applymap(
        af.color_negative_red, subset=pd.IndexSlice[:,[af.FIELD_GOAL_PERCENTAGE_COLUMN,
                                                       af.FREE_THROW_PERCENTAGE_COLUMN,
                                                       af.THREES_MADE_COLUMN, af.POINTS_COLUMN,
                                                       af.REBOUNDS_COLUMN, af.ASSITS_COLUMN,
                                                       af.STEALS_COLUMN, af.BLOCKS_COLUMN
                                                       ]]).applymap(
        af.color_negative_red_tov, subset=pd.IndexSlice[:,[af.TURNOVERS_COLUMN]]).format\
        (COLUMN_9CAT_DECIMAL_FORMAT)
    return colored_matchup_difference

def compare_h2h_team_9cat(team1, team2):
    """
    Function authenticates on Yahoo and creates creates two dataframes containing team roster
    9cat averages for each team specified in the parameters. It calculates the difference between
    both 9cat averages and finally it formats the values for visual ease.
    params team1 (dictionary): team dictionary
    params team2 (dictionary): team dictionary
    return matchup_difference (dataframe): team difference 9cat dataframe
    return colored_matchup_difference (dataframe): formatted team difference 9cat dataframe
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team1_9cat_stats = get_and_format_team_9cat(team1, league)
    team2_9cat_stats = get_and_format_team_9cat(team2, league)
    matchup_difference = team1_9cat_stats.sub(team2_9cat_stats)
    colored_matchup_difference = applymap_color_and_format(matchup_difference)
    return matchup_difference, colored_matchup_difference

In [ ]:
# Get difference dataframes
matchup_difference, colored_matchup_difference = compare_h2h_team_9cat(af.NUNN_OF_YALL_BETTA, af
                                                                       .CRABBEHERBYTHEPUSSY)

In [ ]:
# Difference dataframe
matchup_difference

In [ ]:
# Formatted difference dataframe
colored_matchup_difference

In [ ]:
# Wins/Losses
positive_list = list()
negative_list = list()

for value in matchup_difference.values.tolist()[0]:
    if value > 0:
        positive_list.append(value)
    elif value < 0:
        negative_list.append(abs(value))

In [ ]:
# Print
print(f"Sum of positive list is: {sum(positive_list)}")
print(f"Sum of negative list is: {sum(negative_list)}")

## Free Agent Sort by Rank
This section will work on logic that returns the Leaugue's FA sorted by Rank. This needs to be
done because it will be faster to calculate stats for the top 25 FA rather than all +400 in Yahoo.

In [ ]:
from yahoo_fantasy_api import yhandler
import yahoo_fantasy_api as yfa
yh = yhandler.YHandler(sc)
gm = yfa.Game(sc, 'nba')
# Get League Ids I have access to
league_id_list = gm.league_ids(year=2020)
league_id = "".join(str(id) for id in league_id_list)

In [ ]:
yh.get(f"/league/{league_id}/players;status=FA;sort=AR")

## Get Week Date
This section will create logic needed to return the current week number (i.e. Week 5). This will
allow for the date, start and end, to be found.


In [ ]:
# Authenticate
sc = af.yahoo_fantasy_api_authentication()
league = af.yahoo_fantasy_league(sc)


In [ ]:
# Current week
current_fantasy_week = league.current_week()

# Next week
next_fantasy_week = current_fantasy_week + 1
print(f"Next weeks fantasy mathcup will fall under week {next_fantasy_week}")

In [ ]:
# Get dates
week_start_date = league.week_date_range(current_fantasy_week)[0]

In [ ]:
week_start_date.strftime("%A, %d %B %Y")

## NBA Schedule
This section creates logic to return the NBA schedule. The returned JSON will be filtered by
start and end date in order to return games scheduled in a desired Yahoo Fantasy week.

In [ ]:
import requests
r = requests.get("http://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2020/league/00_full_schedule.json")

In [ ]:
r.json()['lscd']

In [ ]:
import requests

LSCD_KEY = "lscd"
MSCD_KEY = "mscd"
G_KEY = "g"
GID_KEY = "gid"
GAME_ID_KEY = "game_id"
GDATE_KEY = "gdte"
GAME_DATE_KEY= "game_date"
V_KEY = "v"
TID_KEY = "tid"
VISITORS_TEAM_ID_KEY = "visitors_tid"
VISITORS_KEY =  "visitors"
TC_KEY = "tc"
TN_KEY = "tn"
H_KEY = "h"
HOME_TEAM_ID_KEY = "home_tid"
HOME_KEY = "home"

def get_game_information_in_month(nba_json):
    """
    Function loops through a month (n) in nba_json and creates a dataframe containing game
    information, game_id, home/visitor team/team_id and game_date for each month.
    param nba_json (dictionary): dictionary containing game information
    returns game_dataframe (pandas dataframe): dataframe containing game information for a month
    """
    game_dataframe = pd.DataFrame()
    for n in range(len(nba_json)):
        game_dictionary = dict()
        game_dictionary[GID_KEY] = nba_json[n][GID_KEY]
        game_dictionary[GDATE_KEY] = nba_json[n][GDATE_KEY]
        game_dictionary[VISITORS_TEAM_ID_KEY] = nba_json[n][V_KEY][TID_KEY]
        game_dictionary[VISITORS_KEY] = nba_json[n][V_KEY][TC_KEY] + " " \
                                        + nba_json[n][V_KEY][TN_KEY]
        game_dictionary[HOME_TEAM_ID_KEY] = nba_json[n][H_KEY][TID_KEY]
        game_dictionary[HOME_KEY] = nba_json[n][H_KEY][TC_KEY] + " " \
                                    + nba_json[n][H_KEY][TN_KEY]
        game_dataframe = game_dataframe.append(game_dictionary, ignore_index=True)
    return game_dataframe

def get_game_information_in_season(season):
    """
    Function calls the NBA endpoint that returns the season schedule. It then loops through each
    month in that schedule to create a dataframe containing game information, game_id,
    home/visitor team/team_id and game_date for the season.
    param season (string): season for schedule
    returns season_games_dataframe (pandas dataframe): dataframe containing game information for
    a season
    """
    r = requests.get(f"http://data.nba"
                     f".com/data/10s/v2015/json/mobile_teams/nba/{season}/league/00_full_schedule"
                     f".json")
    game_information_json = r.json()[LSCD_KEY]
    season_games_dataframe = pd.DataFrame()
    for n in range(len(game_information_json)):
        month_n_game_dataframe = get_game_information_in_month\
            (game_information_json[n][MSCD_KEY][G_KEY])
        season_games_dataframe = season_games_dataframe.append(month_n_game_dataframe)
    return season_games_dataframe


In [ ]:
season_schedule_2020 = get_game_information_in_season("2020")
season_schedule_2020

## Game Stats in Week
This section will combine the information returned when querying for season schedule with the
logic used to find start/end dates in a Yahoo Fantasy week. The goal is to return the count of
games a specific player plays in a given Yahoo Fantasy week

In [ ]:
# Current week
week_start_date = league.week_date_range(current_fantasy_week)[0]
week_end_date = league.week_date_range(current_fantasy_week)[1]
print(f"The current week start date is: {week_start_date}")
print(f"The current week end date is: {week_end_date}")

In [ ]:
season_schedule_2020["gdte"] = pd.to_datetime(season_schedule_2020["gdte"])
season_schedule_2020.info()

In [ ]:
# Current week games
current_fantasy_games_week = season_schedule_2020[(season_schedule_2020["gdte"] >= \
                                                   str(week_start_date)) &
                                                  (season_schedule_2020["gdte"] <= \
                                                   str(week_end_date))]
current_fantasy_games_week

In [ ]:
current_fantasy_games_week[["home", "visitors"]].value_counts()

In [ ]:
current_fantasy_games_week["visitors"].value_counts()

In [ ]:
def get_week_current_week_information():
    """
    Function finds the current Yahoo Fantasy week number and its start and end date.
    return current_fantasy_week (int): Yahoo Fantasy week number
    return week_start_date (string): start date of Yahoo Fantasy week number
    return week_end_date (string): end date of Yahoo Fantasy week number
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    current_fantasy_week = league.current_week()
    week_start_date = league.week_date_range(current_fantasy_week)[0]
    week_end_date = league.week_date_range(current_fantasy_week)[1]
    return current_fantasy_week, week_start_date, week_end_date

def get_next_week_information():
    """
    Function finds the next Yahoo Fantasy week number and its start and end date.
    return next_fantasy_week (int): Yahoo Fantasy week number
    return week_start_date (string): start date of Yahoo Fantasy week number
    return week_end_date (string): end date of Yahoo Fantasy week number
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    next_fantasy_week = league.current_week() + 1
    week_start_date = league.week_date_range(next_fantasy_week)[0]
    week_end_date = league.week_date_range(next_fantasy_week)[1]
    return next_fantasy_week, week_start_date, week_end_date

def get_player_ids_names_in_fantasy_team(team_dict):
    """

    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team = league.to_team(team_dict[TEAM_KEY])
    team_roster = team.roster()
    player_id_name_team_list = [{PLAYER_ID_KEY : player[PLAYER_ID_KEY], NAME_KEY : player[NAME_KEY]}
                           for player in team_roster]
    player_name_list = [player[NAME_KEY] for player in player_id_name_team_list]
    return player_id_name_team_list, player_name_list

def get_fantasy_week_games_dataframe(season_schedule_dataframe, week_start_date, week_end_date):
    """
    Function filters the entire NBA season schedule on a start/end date to find the scheduled
    games between specific dates. It returns the amount of games each NBA team plays during that
    period of time.
    param season_schedule_dataframe (pandas dataframe): dataframe containing the entire seasons
    NBA schedule
    param week_start_date (string): date to begin filtering by
    param week_start_date (string): date to end filtering by
    return current_fantasy_games_week (pandas dataframe) dataframe containing the scheduled games
    in specified param dates
    return team_game_counts (pandas series): series containing count of games for each NBA team
    """
    filtered_games_week = season_schedule_dataframe[(season_schedule_dataframe[GDATE_KEY] >= \
                                                   str(week_start_date)) &
                                                  (season_schedule_dataframe[GDATE_KEY] <= \
                                                   str(week_end_date))]
    team_game_counts = filtered_games_week[HOME_KEY].append\
        (filtered_games_week[VISITORS_KEY]).value_counts()
    return filtered_games_week, team_game_counts

def get_player_games(team_game_counts_series, player_name_list):
    """
    Function creates a dictionary containing key/value pairs of player names/games in a series of
    NBA team game counts
    param team_game_counts_series (pandas series): series of team name and games playing
    param player_name_list (list): list of player names
    return player_games_dictionary (dictionary): key/value pairing of player names/games playing
    """
    player_games_dictionary = dict()
    team_game_counts_dataframe = pd.DataFrame(team_game_counts_series)
    team_game_counts_dataframe.reset_index(inplace=True)
    team_game_counts_dataframe = team_game_counts_dataframe.replace("LA Clippers", "Los Angeles Clippers")
    for player in player_name_list:
        team = af.yahoo_player_team_and_jersey(player)[0]
        try:
            player_games_dictionary[player] = \
                team_game_counts_dataframe[team_game_counts_dataframe["index"] == team][0].iloc[0]
        except IndexError:
            player_games_dictionary[player] = 0
    return player_games_dictionary

In [ ]:
tm = league.to_team(af.NUNN_OF_YALL_BETTA["team_key"])
team_roster = tm.roster()
player_id_team_list = [{"playerid" : player["player_id"], "playername" : player["name"]} for player in
                       team_roster]
player_id_team_list

In [ ]:
TEAM_KEY = "team"
PLAYER_ID_KEY = "player_id"
NAME_KEY = "name"

def get_player_ids_names_in_fantasy_team(team_dict):
    """

    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team = league.to_team(team_dict[TEAM_KEY])
    team_roster = team.roster()
    player_id_name_team_list = [{PLAYER_ID_KEY : player[PLAYER_ID_KEY], NAME_KEY : player[NAME_KEY]}
                           for player in team_roster]
    player_name_list = [player[NAME_KEY] for player in player_id_name_team_list]
    return player_id_name_team_list, player_name_list

In [ ]:
# Get ids/names in Fantasy team
player_id_name_team_list, player_name_list = get_player_ids_names_in_fantasy_team(af
                                                                                 .NUNN_OF_YALL_BETTA)

In [ ]:
player_name_list

In [ ]:
af.yahoo_player_team_and_jersey("Marcus Smart")[0]


current_fantasy_games_week

In [ ]:
teams_playing_home_visitors = current_fantasy_games_week["home"].append(current_fantasy_games_week["visitors"]).value_counts()

In [ ]:
teams_playing_home_visitors = pd.DataFrame(teams_playing_home_visitors)
teams_playing_home_visitors.reset_index(inplace=True)
teams_playing_home_visitors

In [ ]:
teams_playing_home_visitors = teams_playing_home_visitors.replace("LA Clippers", "Los Angeles Clippers")
teams_playing_home_visitors

In [ ]:
teams_playing_home_visitors[teams_playing_home_visitors["index"] == "Los Angeles Clippers"][0].iloc[0]

In [ ]:
teams_playing_home_visitors.replace(to_replace="LA Clippers", value="Los Angeles Clippers",
                                    regex=True)

In [ ]:
player_games_dictionary = dict()
for player in player_name_list:
    team = af.yahoo_player_team_and_jersey(player)[0]
    player_games_dictionary[player] = teams_playing_home_visitors[team]

In [ ]:
player_games_dictionary

In [ ]:
# Nunn 9Cat averages
nunn_of_yall_betta_team = af.team_9cat_average_stats(af.NUNN_OF_YALL_BETTA, league)

In [ ]:
a = nunn_of_yall_betta_team.copy()
a

In [ ]:
a[a["PLAYER"] == "Marcus Smart"].iloc[:, 3:] * 3

In [ ]:
pd.concat([a[a["PLAYER"] == "Marcus Smart"].iloc[:, :3], (a[a["PLAYER"] == "Marcus Smart"].iloc[:, 3:] * 3)], axis=1)

In [ ]:
PLAYER_COLUMN = "PLAYER"
def get_predicted_player_weekly_9cat(player_games_dictionary, team_9cat_average_stats):
    """
    Function is used to create a dataframe which contains predicted totals for a week of Yahoo
    Fantasy play. For each player, 9cat features are multiplied by the amount of games that
    player plays in one week.
    param player_games_dictionary (dictionary): key/value pairing of player names/games playing
    param team_9cat_average_stats (pandas dataframe): dataframe containing the season average
    9cat stats for each player on a team
    return predicted_9cat_stats (pandas dataframe): dataframe with 9cat averages multiplied by
    games played in a week
    """
    predicted_9cat_stats = pd.DataFrame()
    team_9cat_average_stats = columns_to_dtype_float(team_9cat_average_stats)
    for player in player_games_dictionary.keys():
        player_predicted_9cat_stats = pd.concat(
            [team_9cat_average_stats[team_9cat_average_stats[PLAYER_COLUMN] == player].iloc[:, :3],
            (team_9cat_average_stats[team_9cat_average_stats[PLAYER_COLUMN] == player].iloc[:, 3:]
            * player_games_dictionary[player])], axis=1)
        predicted_9cat_stats = player_predicted_9cat_stats.append(predicted_9cat_stats)
    predicted_9cat_stats.sort_index(inplace=True)
    return predicted_9cat_stats

In [ ]:
predicted_weekly_df = get_predicted_player_weekly_9cat(player_games_dictionary,
                                                    nunn_of_yall_betta_team)
predicted_weekly_df

In [ ]:
TOTAL_ROW = "total"
def get_total_row(roster_dataframe):
    """
    Function creates a row in the provided parameter dataframe which contains the total for
    categories that should be summed and averages for categories that should be averaged
    param roster_dataframe (pandas dataframe): dataframe containing player week 9cat stats
    return roster_dataframe_with_total (pandas dataframe): dataframe containing total row
    """
    sum_9cat_column_dataframe = pd.DataFrame(roster_dataframe.iloc[:, 3:].sum()).T
    mean_9cat_column_dataframe =  pd.DataFrame(roster_dataframe.iloc[:, 1:3].mean()).T
    total_dataframe = pd.concat([mean_9cat_column_dataframe,  sum_9cat_column_dataframe], axis=1)
    total_dataframe[PLAYER_COLUMN] = TOTAL_ROW
    roster_dataframe_with_total = roster_dataframe.append(total_dataframe)
    roster_dataframe_with_total.reset_index(inplace=True, drop=True)
    return roster_dataframe_with_total

In [ ]:
b = get_total_row(predicted_weekly_df)



In [ ]:
af.format_dataframe_decimals(b)



In [ ]:
def get_teams_weekly_predicted_stats_pipeline(team_dict, week="next", season=2020,
                                              team_9cat_average_stats_data=None):
    """
    Function pipelines the process required to find a league teams weekly predicted 9cat totals.
    Initially, the number of games for each player in the provided parameter week is found.
    Average 9cat player stats are multiplied by that number and the entire roster stats are
    summed to provide a total. The dataframe is finally formatted.
    param team_dict (dictionary): dictionary of team from league
    param week (string): determines which week to calculate weekly games for
    param season (int): determines which season to get NBA schedule for
    param team_9cat_average_stats_data (pandas dataframe): dataframe containing player 9cat season
    averages
    return roster_dataframe_with_total (pandas dataframe): dataframe containing player 9cat
    predicted weekly stats and entire roster totals
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    if week == "next":
        fantasy_week, week_start_date, week_end_date = get_next_week_information()
    elif week == "current":
        fantasy_week, week_start_date, week_end_date = get_week_current_week_information()
    else:
        raise ValueError("Parameter 'week must be 'current' or 'next'.")
    season_games_dataframe = get_game_information_in_season(season)
    filtered_games_week, team_game_counts = get_fantasy_week_games_dataframe\
        (season_games_dataframe, week_start_date, week_end_date)
    player_id_name_team_list, player_name_list = get_player_ids_names_in_fantasy_team(team_dict)
    player_games_dictionary = get_player_games(team_game_counts, player_name_list)
    if team_9cat_average_stats_data is None:
        team_9cat_average_stats_dataframe = af.team_9cat_average_stats(team_dict, league)
        predicted_weekly_9cat_stats = get_predicted_player_weekly_9cat(player_games_dictionary,
                                                                 team_9cat_average_stats_dataframe)
    else:
        predicted_weekly_9cat_stats = get_predicted_player_weekly_9cat(player_games_dictionary,
                                                                 team_9cat_average_stats_data)
    roster_dataframe_with_total = get_total_row(predicted_weekly_9cat_stats)
    roster_dataframe_with_total = af.format_dataframe_decimals(roster_dataframe_with_total)
    return roster_dataframe_with_total

In [ ]:
get_teams_weekly_predicted_stats_pipeline(af.NUNN_OF_YALL_BETTA, week="current")

### All Together


In [ ]:
import requests
import pandas as pd
from src.data import all_functions as af

PLAYER_COLUMN = "PLAYER"
TEAM_KEY = "team_key"
PLAYER_ID_KEY = "player_id"
NAME_KEY = "name"
MEAN_ROW = "mean"
NAME_KEY = "name"
LSCD_KEY = "lscd"
MSCD_KEY = "mscd"
G_KEY = "g"
GID_KEY = "gid"
GAME_ID_KEY = "game_id"
GDATE_KEY = "gdte"
GAME_DATE_KEY= "game_date"
V_KEY = "v"
TID_KEY = "tid"
VISITORS_TEAM_ID_KEY = "visitors_tid"
VISITORS_KEY =  "visitors"
TC_KEY = "tc"
TN_KEY = "tn"
H_KEY = "h"
HOME_TEAM_ID_KEY = "home_tid"
HOME_KEY = "home"
TOTAL_ROW = "total"
INDEX_COLUMN = "index"
NBA_NAME_LA_CLIPPERS = "LA Clippers"
YAHOO_NAME_LA_CLIPPERS = "Los Angeles Clippers"
COLUMN_9CAT_DECIMAL_FORMAT = {af.FIELD_GOAL_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.FREE_THROW_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.THREES_MADE_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.POINTS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.REBOUNDS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.ASSITS_COLUMN : af.TWO_DECIMAL_FORMAT, af.STEALS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.BLOCKS_COLUMN : af.TWO_DECIMAL_FORMAT, af.TURNOVERS_COLUMN : af.TWO_DECIMAL_FORMAT}

def columns_to_dtype_float(dataframe):
    """
    Converts dataframe columns to dtype float.
    param dataframe (pandas dataframe): dataframe to change column types
    returns dataframe (pandas dataframe): dataframe with changed column types
    """
    for column in dataframe.columns:
        try:
            dataframe[column] = dataframe[column].astype(float)
        except ValueError:
            pass
    return dataframe

def get_and_format_team_9cat(team, league):
    """
    Function first finds the teams 9cat averages by calling the *team_9cat_average_stats*
    function. It then filters to find the team mean and cleans and formats this single row
    dataframe. It changes the dtype of the columns to be floats.
    param team (dictionary): team dictionary
    param league (yahoo oauth object): yahoo authentication object
    returns team_9cat (dataframe): dataframe of team average 9cats
    """
    team_9cat = af.team_9cat_average_stats(team, league)
    team_9cat = team_9cat.loc[MEAN_ROW]
    team_9cat[PLAYER_COLUMN] = team[NAME_KEY]
    team_9cat = pd.DataFrame(team_9cat).T
    team_9cat.drop(PLAYER_COLUMN, inplace=True, axis=1)
    team_9cat = columns_to_dtype_float(team_9cat)
    return team_9cat

def applymap_color_and_format(dataframe):
    """
    Function takes a dataframe containing 9cat stat columns and returns a color coded and decimal
    place formatted dataframe for visual ease.
    param dataframe (pandas dataframe): dataframe to format
    returns dataframe (pandas dataframe): dataframe with formatted and colored values
    """
    colored_matchup_difference = dataframe.style.applymap(
        af.color_negative_red, subset=pd.IndexSlice[:,[af.FIELD_GOAL_PERCENTAGE_COLUMN,
                                                       af.FREE_THROW_PERCENTAGE_COLUMN,
                                                       af.THREES_MADE_COLUMN, af.POINTS_COLUMN,
                                                       af.REBOUNDS_COLUMN, af.ASSITS_COLUMN,
                                                       af.STEALS_COLUMN, af.BLOCKS_COLUMN
                                                       ]]).applymap(
        af.color_negative_red_tov, subset=pd.IndexSlice[:,[af.TURNOVERS_COLUMN]]).format\
        (COLUMN_9CAT_DECIMAL_FORMAT)
    return colored_matchup_difference

def compare_h2h_team_9cat(team1, team2):
    """
    Function authenticates on Yahoo and creates creates two dataframes containing team roster
    9cat averages for each team specified in the parameters. It calculates the difference between
    both 9cat averages and finally it formats the values for visual ease.
    params team1 (dictionary): team dictionary
    params team2 (dictionary): team dictionary
    return matchup_difference (dataframe): team difference 9cat dataframe
    return colored_matchup_difference (dataframe): formatted team difference 9cat dataframe
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team1_9cat_stats = get_and_format_team_9cat(team1, league)
    team2_9cat_stats = get_and_format_team_9cat(team2, league)
    matchup_difference = team1_9cat_stats.sub(team2_9cat_stats)
    colored_matchup_difference = applymap_color_and_format(matchup_difference)
    return matchup_difference, colored_matchup_difference

def get_game_information_in_month(nba_json):
    """
    Function loops through a month (n) in nba_json and creates a dataframe containing game
    information, game_id, home/visitor team/team_id and game_date for each month.
    param nba_json (dictionary): dictionary containing game information
    returns game_dataframe (pandas dataframe): dataframe containing game information for a month
    """
    game_dataframe = pd.DataFrame()
    for n in range(len(nba_json)):
        game_dictionary = dict()
        game_dictionary[GID_KEY] = nba_json[n][GID_KEY]
        game_dictionary[GDATE_KEY] = nba_json[n][GDATE_KEY]
        game_dictionary[VISITORS_TEAM_ID_KEY] = nba_json[n][V_KEY][TID_KEY]
        game_dictionary[VISITORS_KEY] = nba_json[n][V_KEY][TC_KEY] + " " \
                                        + nba_json[n][V_KEY][TN_KEY]
        game_dictionary[HOME_TEAM_ID_KEY] = nba_json[n][H_KEY][TID_KEY]
        game_dictionary[HOME_KEY] = nba_json[n][H_KEY][TC_KEY] + " " \
                                    + nba_json[n][H_KEY][TN_KEY]
        game_dataframe = game_dataframe.append(game_dictionary, ignore_index=True)
    return game_dataframe

def get_game_information_in_season(season):
    """
    Function calls the NBA endpoint that returns the season schedule. It then loops through each
    month in that schedule to create a dataframe containing game information, game_id,
    home/visitor team/team_id and game_date for the season.
    param season (string): season for schedule
    returns season_games_dataframe (pandas dataframe): dataframe containing game information for
    a season
    """
    r = requests.get(f"http://data.nba"
                     f".com/data/10s/v2015/json/mobile_teams/nba/{season}/league/00_full_schedule"
                     f".json")
    game_information_json = r.json()[LSCD_KEY]
    season_games_dataframe = pd.DataFrame()
    for n in range(len(game_information_json)):
        month_n_game_dataframe = get_game_information_in_month\
            (game_information_json[n][MSCD_KEY][G_KEY])
        season_games_dataframe = season_games_dataframe.append(month_n_game_dataframe)
    return season_games_dataframe

def get_week_current_week_information():
    """
    Function finds the current Yahoo Fantasy week number and its start and end date.
    return current_fantasy_week (int): Yahoo Fantasy week number
    return week_start_date (string): start date of Yahoo Fantasy week number
    return week_end_date (string): end date of Yahoo Fantasy week number
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    current_fantasy_week = league.current_week()
    week_start_date = league.week_date_range(current_fantasy_week)[0]
    week_end_date = league.week_date_range(current_fantasy_week)[1]
    return current_fantasy_week, week_start_date, week_end_date

def get_next_week_information():
    """
    Function finds the next Yahoo Fantasy week number and its start and end date.
    return next_fantasy_week (int): Yahoo Fantasy week number
    return week_start_date (string): start date of Yahoo Fantasy week number
    return week_end_date (string): end date of Yahoo Fantasy week number
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    next_fantasy_week = league.current_week() + 1
    week_start_date = league.week_date_range(next_fantasy_week)[0]
    week_end_date = league.week_date_range(next_fantasy_week)[1]
    return next_fantasy_week, week_start_date, week_end_date

def get_player_ids_names_in_fantasy_team(team_dict):
    """
    Function finds the Yahoo Fantasy player ids and player names rostered by a team in the league
    param team_dict (dictionary): dictionary containing information about a owners team
    return player_id_name_team_list (list): Yahoo Fantasy player ids and names
    return player_name_list (list): Yahoo Fantasy player names
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team = league.to_team(team_dict[TEAM_KEY])
    team_roster = team.roster()
    player_id_name_team_list = [{PLAYER_ID_KEY : player[PLAYER_ID_KEY], NAME_KEY : player[NAME_KEY]}
                           for player in team_roster]
    player_name_list = [player[NAME_KEY] for player in player_id_name_team_list]
    return player_id_name_team_list, player_name_list

def get_fantasy_week_games_dataframe(season_schedule_dataframe, week_start_date, week_end_date):
    """
    Function filters the entire NBA season schedule on a start/end date to find the scheduled
    games between specific dates. It returns the amount of games each NBA team plays during that
    period of time.
    param season_schedule_dataframe (pandas dataframe): dataframe containing the entire seasons
    NBA schedule
    param week_start_date (string): date to begin filtering by
    param week_start_date (string): date to end filtering by
    return current_fantasy_games_week (pandas dataframe) dataframe containing the scheduled games
    in specified param dates
    return team_game_counts (pandas series): series containing count of games for each NBA team
    """
    filtered_games_week = season_schedule_dataframe[(season_schedule_dataframe[GDATE_KEY] >=
                                                   str(week_start_date)) &
                                                  (season_schedule_dataframe[GDATE_KEY] <=
                                                   str(week_end_date))]
    team_game_counts = filtered_games_week[HOME_KEY].append\
        (filtered_games_week[VISITORS_KEY]).value_counts()
    return filtered_games_week, team_game_counts

def get_player_games(team_game_counts_series, player_name_list):
    """
    Function creates a dictionary containing key/value pairs of player names/games in a series of
    NBA team game counts
    param team_game_counts_series (pandas series): series of team name and games playing
    param player_name_list (list): list of player names
    return player_games_dictionary (dictionary): key/value pairing of player names/games playing
    """
    player_games_dictionary = dict()
    team_game_counts_dataframe = pd.DataFrame(team_game_counts_series)
    team_game_counts_dataframe.reset_index(inplace=True)
    team_game_counts_dataframe = team_game_counts_dataframe.replace(NBA_NAME_LA_CLIPPERS,
                                                                    YAHOO_NAME_LA_CLIPPERS)
    for player in player_name_list:
        team = af.yahoo_player_team_and_jersey(player)[0]
        try:
            player_games_dictionary[player] = \
                team_game_counts_dataframe[team_game_counts_dataframe["index"] == team][0].iloc[0]
        except IndexError:
            player_games_dictionary[player] = 0
    return player_games_dictionary

def get_predicted_player_weekly_9cat(player_games_dictionary, team_9cat_average_stats):
    """
    Function is used to create a dataframe which contains predicted totals for a week of Yahoo
    Fantasy play. For each player, 9cat features are multiplied by the amount of games that
    player plays in one week.
    param player_games_dictionary (dictionary): key/value pairing of player names/games playing
    param team_9cat_average_stats (pandas dataframe): dataframe containing the season average
    9cat stats for each player on a team
    return predicted_9cat_stats (pandas dataframe): dataframe with 9cat averages multiplied by
    games played in a week
    """
    predicted_9cat_stats = pd.DataFrame()
    team_9cat_average_stats = columns_to_dtype_float(team_9cat_average_stats)
    for player in player_games_dictionary.keys():
        player_predicted_9cat_stats = pd.concat(
            [team_9cat_average_stats[team_9cat_average_stats[PLAYER_COLUMN] == player].iloc[:, :3],
            (team_9cat_average_stats[team_9cat_average_stats[PLAYER_COLUMN] == player].iloc[:, 3:]
            * player_games_dictionary[player])], axis=1)
        predicted_9cat_stats = player_predicted_9cat_stats.append(predicted_9cat_stats)
    predicted_9cat_stats.sort_index(inplace=True)
    return predicted_9cat_stats

def get_total_row(roster_dataframe):
    """
    Function creates a row in the provided parameter dataframe which contains the total for
    categories that should be summed and averages for categories that should be averaged
    param roster_dataframe (pandas dataframe): dataframe containing player week 9cat stats
    return roster_dataframe_with_total (pandas dataframe): dataframe containing total row
    """
    sum_9cat_column_dataframe = pd.DataFrame(roster_dataframe.iloc[:, 3:].sum()).T
    mean_9cat_column_dataframe =  pd.DataFrame(roster_dataframe.iloc[:, 1:3].mean()).T
    total_dataframe = pd.concat([mean_9cat_column_dataframe,  sum_9cat_column_dataframe], axis=1)
    total_dataframe[PLAYER_COLUMN] = TOTAL_ROW
    roster_dataframe_with_total = roster_dataframe.append(total_dataframe)
    roster_dataframe_with_total.reset_index(inplace=True, drop=True)
    return roster_dataframe_with_total

def get_teams_weekly_predicted_stats_pipeline(team_dict, week="next", season=2020,
                                              team_9cat_average_stats_data=None):
    """
    Function pipelines the process required to find a league teams weekly predicted 9cat totals.
    Initially, the number of games for each player in the provided parameter week is found.
    Average 9cat player stats are multiplied by that number and the entire roster stats are
    summed to provide a total. The dataframe is finally formatted.
    param team_dict (dictionary): dictionary of team from league
    param week (string): determines which week to calculate weekly games for
    param season (int): determines which season to get NBA schedule for
    param team_9cat_average_stats_data (pandas dataframe): dataframe containing player 9cat season
    averages
    return roster_dataframe_with_total (pandas dataframe): dataframe containing player 9cat
    predicted weekly stats and entire roster totals
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    if week == "next":
        fantasy_week, week_start_date, week_end_date = get_next_week_information()
    elif week == "current":
        fantasy_week, week_start_date, week_end_date = get_week_current_week_information()
    else:
        raise ValueError("Parameter 'week' must be 'current' or 'next'.")
    season_games_dataframe = get_game_information_in_season(season)
    filtered_games_week, team_game_counts = get_fantasy_week_games_dataframe\
        (season_games_dataframe, week_start_date, week_end_date)
    player_id_name_team_list, player_name_list = get_player_ids_names_in_fantasy_team(team_dict)
    player_games_dictionary = get_player_games(team_game_counts, player_name_list)
    if team_9cat_average_stats_data is None:
        team_9cat_average_stats_dataframe = af.team_9cat_average_stats(team_dict, league)
        predicted_weekly_9cat_stats = get_predicted_player_weekly_9cat(player_games_dictionary,
                                                                 team_9cat_average_stats_dataframe)
    else:
        predicted_weekly_9cat_stats = get_predicted_player_weekly_9cat(player_games_dictionary,
                                                                 team_9cat_average_stats_data)
    roster_dataframe_with_total = get_total_row(predicted_weekly_9cat_stats)
    roster_dataframe_with_total = af.format_dataframe_decimals(roster_dataframe_with_total)
    return roster_dataframe_with_total

def get_filtered_total_row(total_dataframe, team_dict):
    """
    Function filters a dataframe containing weekly total 9cat stats and returns the *total* row
    so that it can be used for differencing. The function appends the team name to the total row
    param total_dataframe (pandas dataframe): dataframe containing weekly total 9cat stats
    param team_dict (dictionary): dictionary of team from league
    return total_row_dataframe (pandas dataframe): single row dataframe containing total 9cat
    stats with team name
    return total_row_dataframe_no_name (pandas dataframe): single row dataframe containing total 9cat
    stats without team name
    """
    total_row_dataframe = total_dataframe[total_dataframe[PLAYER_COLUMN] == TOTAL_ROW]
    total_row_dataframe[PLAYER_COLUMN] = team_dict[NAME_KEY]
    total_row_dataframe.index = [TOTAL_ROW]
    total_row_dataframe_no_name = total_row_dataframe.drop(PLAYER_COLUMN, axis=1)
    return total_row_dataframe, total_row_dataframe_no_name

def weekly_matchup_evaluator(team1_dict, team2_dict, team1_9cat_average_stats_data=None,
                             team2_9cat_average_stats_data=None, week="next", season=2020):
    """

    """
    team1_roster_dataframe_with_total = get_teams_weekly_predicted_stats_pipeline(
        team1_dict, week=week, season=season,
        team_9cat_average_stats_data=team1_9cat_average_stats_data)
    team2_roster_dataframe_with_total = get_teams_weekly_predicted_stats_pipeline(
        team2_dict, week=week, season=season,
        team_9cat_average_stats_data=team2_9cat_average_stats_data)
    team1_total_row_dataframe, team1_total_row_dataframe_no_name = get_filtered_total_row\
        (team1_roster_dataframe_with_total, team1_dict)
    team2_total_row_dataframe, team2_total_row_dataframe_no_name = get_filtered_total_row\
        (team2_roster_dataframe_with_total, team2_dict)
    team1_total_row_dataframe_no_name = columns_to_dtype_float(team1_total_row_dataframe_no_name)
    team2_total_row_dataframe_no_name = columns_to_dtype_float(team2_total_row_dataframe_no_name)
    total_difference_dataframe = team1_total_row_dataframe_no_name.subtract\
        (team2_total_row_dataframe_no_name)
    total_difference_dataframe_colored = applymap_color_and_format(total_difference_dataframe)
    return total_difference_dataframe, total_difference_dataframe_colored

In [ ]:
nunn_of_yall_betta_weekly_stats = get_teams_weekly_predicted_stats_pipeline(af.NUNN_OF_YALL_BETTA,
                                                               week="current")
nunn_of_yall_betta_weekly_stats

In [ ]:
magic_johnson_stats = get_teams_weekly_predicted_stats_pipeline(af.MAGICS_JOHNSON,
                                                               week="current")
magic_johnson_stats

In [ ]:
nyb_total_row_dataframe, nyb_total_row_dataframe_no_name = get_filtered_total_row\
    (nunn_of_yall_betta_weekly_stats, af.NUNN_OF_YALL_BETTA)

mjs_total_row_dataframe, mjs_total_row_dataframe_no_name = get_filtered_total_row\
    (magic_johnson_stats, af.MAGICS_JOHNSON)


In [ ]:
nyb_total_row_dataframe_no_name = columns_to_dtype_float(nyb_total_row_dataframe_no_name)
mjs_total_row_dataframe_no_name = columns_to_dtype_float(mjs_total_row_dataframe_no_name)

total_difference_dataframe = nyb_total_row_dataframe_no_name.subtract\
    (mjs_total_row_dataframe_no_name)


In [ ]:
applymap_color_and_format(total_difference_dataframe)

In [ ]:
def weekly_matchup_evaluator(team1_dict, team2_dict, team1_9cat_average_stats_data=None,
                             team2_9cat_average_stats_data=None, week="next", season=2020):
    """

    """
    team1_roster_dataframe_with_total = get_teams_weekly_predicted_stats_pipeline(
        team1_dict, week=week, season=season,
        team_9cat_average_stats_data=team1_9cat_average_stats_data)
    team2_roster_dataframe_with_total = get_teams_weekly_predicted_stats_pipeline(
        team2_dict, week=week, season=season,
        team_9cat_average_stats_data=team2_9cat_average_stats_data)
    team1_total_row_dataframe, team1_total_row_dataframe_no_name = get_filtered_total_row\
        (team1_roster_dataframe_with_total, team1_dict)
    team2_total_row_dataframe, team2_total_row_dataframe_no_name = get_filtered_total_row\
        (team2_roster_dataframe_with_total, team2_dict)
    team1_total_row_dataframe_no_name = columns_to_dtype_float(team1_total_row_dataframe_no_name)
    team2_total_row_dataframe_no_name = columns_to_dtype_float(team2_total_row_dataframe_no_name)
    total_difference_dataframe = team1_total_row_dataframe_no_name.subtract\
        (team2_total_row_dataframe_no_name)
    total_difference_dataframe_colored = applymap_color_and_format(total_difference_dataframe)
    return total_difference_dataframe, total_difference_dataframe_colored

In [ ]:
total_difference_dataframe, total_difference_dataframe_colored = weekly_matchup_evaluator(
    af.NUNN_OF_YALL_BETTA, af.MAGICS_JOHNSON, team1_9cat_average_stats_data=None,
    team2_9cat_average_stats_data=None, week="current", season=2020)

In [ ]:
total_difference_dataframe

## Current Week Matchup Stats
This section will use the Yahoo Fantasy API to check on the current matchup

In [ ]:
# Authenticate
sc = af.yahoo_fantasy_api_authentication()
league = af.yahoo_fantasy_league(sc)

In [ ]:
matchups_w8 = league.matchups(week=8)

In [ ]:
FANTASY_CONTENT_KEY = "fantasy_content"
LEAGUE_KEY = "league"
SCOREBOARD_KEY = "scoreboard"
ZERO_KEY = "0"
MATCHUPS_KEY = "matchups"
MATCHUP_KEY = "matchup"
TEAMS_KEY = "teams"
ONE_KEY = "1"
TEAM_KEY = "team"
NAME_KEY = "name"

def get_teams_matchup_number(team_dict, matchups):
    """
    Function loops through returned the returned Yahoo Fantasy matchup nested dictionary to find
    the matchup number of the specified parameter team name.
    param team_dict (dictionary): dictionary containing information about a owners team
    param matchups (dictionary): league matchup information from specified week number
    return matchup (string): string value for the index of the specified teams nested matchup
    dictionary in the original matchup dictionary
    """
    for matchup in range(len(matchups[FANTASY_CONTENT_KEY][LEAGUE_KEY][1][SCOREBOARD_KEY][ZERO_KEY]\
                [MATCHUPS_KEY]) - 1):
        if matchups[FANTASY_CONTENT_KEY][LEAGUE_KEY][1][SCOREBOARD_KEY][ZERO_KEY]\
                [MATCHUPS_KEY][str(matchup)][MATCHUP_KEY][ZERO_KEY][TEAMS_KEY][ZERO_KEY]\
                [TEAM_KEY][0][2][NAME_KEY] == team_dict[NAME_KEY] \
                or matchups[FANTASY_CONTENT_KEY][LEAGUE_KEY][1][SCOREBOARD_KEY][ZERO_KEY]\
                [MATCHUPS_KEY][str(matchup)][MATCHUP_KEY][ZERO_KEY][TEAMS_KEY][ONE_KEY]\
                [TEAM_KEY][0][2][NAME_KEY] == team_dict[NAME_KEY]:
            return str(matchup)

In [ ]:
get_teams_matchup_number(af.NUNN_OF_YALL_BETTA, matchups_w8)

# With matchup number create stats dataframe for both teams in matchup

In [ ]:
def get_team_name_from_matchup(matchups, matchup_number, matchup_team_number):
    """
    Function returns the team name in a matchup.
    param matchups (dictionary): league matchup information from specified week number
    param matchup_number (string): matchup number for in matchups dictionary
    param matchup_team_number (string): either "0" or "1" indicating matchup team number
    return team_name (string): name of team in matchup
    """
    team_name = matchups[FANTASY_CONTENT_KEY][LEAGUE_KEY][1][SCOREBOARD_KEY][ZERO_KEY]\
                [MATCHUPS_KEY][matchup_number][MATCHUP_KEY][ZERO_KEY][TEAMS_KEY]\
                [matchup_team_number][TEAM_KEY][0][2][NAME_KEY]
    return team_name

In [ ]:
TEAM_STATS_KEY = "team_stats" 
STATS_KEY = "stats"

def get_live_matchup_stats_dictionary(matchups, matchup_number, matchup_team_number):
    """
    Function returns a dictionary containing 9cat stats plus two more for a given matchup and
    matchup team number.
    param matchups (dictionary): league matchup information from specified week number
    param matchup_number (string): matchup number for in matchups dictionary
    param matchup_team_number (string): either "0" or "1" indicating matchup team number
    return live_matchup_stats (dictionary): dictionary of 9cat stats plus two more stats
    """
    live_matchup_stats = matchups[FANTASY_CONTENT_KEY][LEAGUE_KEY][1][SCOREBOARD_KEY][ZERO_KEY]\
                [MATCHUPS_KEY][matchup_number][MATCHUP_KEY][ZERO_KEY][TEAMS_KEY]\
                [matchup_team_number][TEAM_KEY][1][TEAM_STATS_KEY][STATS_KEY]
    return live_matchup_stats


In [ ]:
STAT_KEY = "stat"
VALUE_KEY = "value"
MATCHUP_COLUMNS = [af.PLAYER_COLUMN] + af.NINE_CAT_COLUMNS

def get_dataframe_from_live_matchup_stats(matchup_stats_dictionary, team_name):
    """
    Function creates a 1x10 dataframe containing the name of the team along with this live 9cat
    stats for a weeks matchup.
    param matchup_stats_dictionary (dictionary): dictionary containing the stats for a team
    param team_name (string): the name of the team
    return live_matchup_9cat_stats_dataframe (pandas dataframe): dataframe containing the name of the team along with this live 9cat
    stats for a weeks matchup
    """
    matchup_9cat_stats_list = list()
    for index in range(len(matchup_stats_dictionary)):
        if index != 0 and index != 2:
            matchup_9cat_stats_list.append(matchup_stats_dictionary[index][STAT_KEY][VALUE_KEY])
    values = [team_name] + matchup_9cat_stats_list
    live_matchup_9cat_stats_dataframe = pd.DataFrame(values).T
    live_matchup_9cat_stats_dataframe.columns = MATCHUP_COLUMNS
    return live_matchup_9cat_stats_dataframe

In [ ]:
matchups_w8 = league.matchups(week=8)
matchup_number = get_teams_matchup_number(af.NUNN_OF_YALL_BETTA, matchups_w8)
team_name = get_team_name_from_matchup(matchups_w8, matchup_number, "0")
team_stats_dictionary = get_live_matchup_stats_dictionary(matchups_w8, matchup_number,
                                                       "0")
get_dataframe_from_live_matchup_stats(team_stats_dictionary, team_name)
team_stats_dictionary

In [ ]:
def get_matchup_teams_9cat_stat_dataframes(week, team_dict):
    """

    @param week:
    @type week:
    @param team_dict:
    @type team_dict:
    @return:
    @rtype:
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    matchups = league.matchups(week=week)
    matchup_number = get_teams_matchup_number(team_dict, matchups)
    first_team_name = get_team_name_from_matchup(matchups_w8, matchup_number, "0")
    second_team_name = get_team_name_from_matchup(matchups_w8, matchup_number, "1")
    first_team_stats_dictionary = get_live_matchup_stats_dictionary(matchups_w8, matchup_number,
                                                                    "0")
    second_team_stats_dictionary = get_live_matchup_stats_dictionary(matchups_w8, matchup_number,
                                                                     "1")
    first_team_stats_dataframe = get_dataframe_from_live_matchup_stats(first_team_stats_dictionary,
                                                              first_team_name)
    second_team_stats_dataframe = get_dataframe_from_live_matchup_stats(second_team_stats_dictionary,
                                                              second_team_name)
    return first_team_stats_dataframe, second_team_stats_dataframe


In [ ]:
FANTASY_CONTENT_KEY = "fantasy_content"
LEAGUE_KEY = "league"
SCOREBOARD_KEY = "scoreboard"
ZERO_KEY = "0"
MATCHUPS_KEY = "matchups"
MATCHUP_KEY = "matchup"
TEAMS_KEY = "teams"
ONE_KEY = "1"
TEAM_KEY_2 = "team"
NAME_KEY = "name"
TEAM_STATS_KEY = "team_stats"
STATS_KEY = "stats"
STAT_KEY = "stat"
VALUE_KEY = "value"
MATCHUP_COLUMNS = [af.PLAYER_COLUMN] + af.NINE_CAT_COLUMNS

def get_teams_matchup_number(team_dict, matchups):
    """
    Function loops through returned the returned Yahoo Fantasy matchup nested dictionary to find
    the matchup number of the specified parameter team name.
    param team_dict (dictionary): dictionary containing information about a owners team
    param matchups (dictionary): league matchup information from specified week number
    return matchup (string): string value for the index of the specified teams nested matchup
    dictionary in the original matchup dictionary
    """
    for matchup in range(len(matchups[FANTASY_CONTENT_KEY][LEAGUE_KEY][1][SCOREBOARD_KEY][ZERO_KEY]\
                [MATCHUPS_KEY]) - 1):
        if matchups[FANTASY_CONTENT_KEY][LEAGUE_KEY][1][SCOREBOARD_KEY][ZERO_KEY]\
                [MATCHUPS_KEY][str(matchup)][MATCHUP_KEY][ZERO_KEY][TEAMS_KEY][ZERO_KEY]\
                [TEAM_KEY_2][0][2][NAME_KEY] == team_dict[NAME_KEY] \
                or matchups[FANTASY_CONTENT_KEY][LEAGUE_KEY][1][SCOREBOARD_KEY][ZERO_KEY]\
                [MATCHUPS_KEY][str(matchup)][MATCHUP_KEY][ZERO_KEY][TEAMS_KEY][ONE_KEY]\
                [TEAM_KEY_2][0][2][NAME_KEY] == team_dict[NAME_KEY]:
            return str(matchup)

def get_team_name_from_matchup(matchups, matchup_number, matchup_team_number):
    """
    Function returns the team name in a matchup.
    param matchups (dictionary): league matchup information from specified week number
    param matchup_number (string): matchup number for in matchups dictionary
    param matchup_team_number (string): either "0" or "1" indicating matchup team number
    return team_name (string): name of team in matchup
    """
    team_name = matchups[FANTASY_CONTENT_KEY][LEAGUE_KEY][1][SCOREBOARD_KEY][ZERO_KEY]\
                [MATCHUPS_KEY][matchup_number][MATCHUP_KEY][ZERO_KEY][TEAMS_KEY]\
                [matchup_team_number][TEAM_KEY_2][0][2][NAME_KEY]
    return team_name

def get_live_matchup_stats_dictionary(matchups, matchup_number, matchup_team_number):
    """
    Function returns a dictionary containing 9cat stats plus two more for a given matchup and
    matchup team number.
    param matchups (dictionary): league matchup information from specified week number
    param matchup_number (string): matchup number for in matchups dictionary
    param matchup_team_number (string): either "0" or "1" indicating matchup team number
    return live_matchup_stats (dictionary): dictionary of 9cat stats plus two more stats
    """
    live_matchup_stats = matchups[FANTASY_CONTENT_KEY][LEAGUE_KEY][1][SCOREBOARD_KEY][ZERO_KEY]\
                [MATCHUPS_KEY][matchup_number][MATCHUP_KEY][ZERO_KEY][TEAMS_KEY]\
                [matchup_team_number][TEAM_KEY_2][1][TEAM_STATS_KEY][STATS_KEY]
    return live_matchup_stats

def get_dataframe_from_live_matchup_stats(matchup_stats_dictionary, team_name):
    """
    Function creates a 1x10 dataframe containing the name of the team along with this live 9cat
    stats for a weeks matchup.
    param matchup_stats_dictionary (dictionary): dictionary containing the stats for a team
    param team_name (string): the name of the team
    return live_matchup_9cat_stats_dataframe (pandas dataframe): dataframe containing the name of the team along with this live 9cat
    stats for a weeks matchup
    """
    matchup_9cat_stats_list = list()
    for index in range(len(matchup_stats_dictionary)):
        if index != 0 and index != 2:
            matchup_9cat_stats_list.append(matchup_stats_dictionary[index][STAT_KEY][VALUE_KEY])
    values = [team_name] + matchup_9cat_stats_list
    live_matchup_9cat_stats_dataframe = pd.DataFrame(values).T
    live_matchup_9cat_stats_dataframe.columns = MATCHUP_COLUMNS
    return live_matchup_9cat_stats_dataframe

# Format and find 9cat differences
def remove_player_and_float_convert(dataframe):
    """
    Function cleans dataframe by converting all columns to dtype float
    @param dataframe: dataframe with columns to convert
    @return: converted dataframe
    """
    for column in dataframe.columns:
        dataframe[column] = dataframe[column].astype(float)
    return dataframe

def format_3ptm_to_tov_as_int(dataframe):
    """
    Function changes the 3ptm to tov columns to dtype int needed for the live score dataframes.
    @param dataframe: dataframe with columns to convert
    @return: converted dataframe
    """
    for column in dataframe.columns[2:]:
        dataframe[column] = dataframe[column].astype(int)
    return dataframe

def get_matchup_teams_9cat_stat_dataframes(week, team_dict):
    """

    @param week:
    @type week:
    @param team_dict:
    @type team_dict:
    @return:
    @rtype:
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    matchups = league.matchups(week=week)
    matchup_number = get_teams_matchup_number(team_dict, matchups)
    first_team_name = get_team_name_from_matchup(matchups, matchup_number, "0")
    second_team_name = get_team_name_from_matchup(matchups, matchup_number, "1")
    first_team_stats_dictionary = get_live_matchup_stats_dictionary(matchups, matchup_number,
                                                                    "0")
    second_team_stats_dictionary = get_live_matchup_stats_dictionary(matchups, matchup_number,
                                                                     "1")
    first_team_stats_dataframe = get_dataframe_from_live_matchup_stats(first_team_stats_dictionary,
                                                              first_team_name)
    first_team_stats_dataframe = first_team_stats_dataframe.set_index(PLAYER_COLUMN)
    second_team_stats_dataframe = get_dataframe_from_live_matchup_stats(second_team_stats_dictionary,
                                                              second_team_name)
    second_team_stats_dataframe = second_team_stats_dataframe.set_index(PLAYER_COLUMN)
    matchup_stats_dataframe = first_team_stats_dataframe.append(second_team_stats_dataframe)
    matchup_stats_dataframe = remove_player_and_float_convert(matchup_stats_dataframe)
    matchup_stats_dataframe = format_3ptm_to_tov_as_int(matchup_stats_dataframe)
    return first_team_name, first_team_stats_dataframe, second_team_name, \
           second_team_stats_dataframe, matchup_stats_dataframe

def get_team_dict_from_team_name(team_name):
    """
    Function returns the team dictionary from a team name.
    @param team_name: name of fantasy team
    @type team_name: string
    @return: dictionary of fantasy team
    @rtype: dicitonary
    """
    if team_name == af.AUTOPICK[NAME_KEY]:
        team_dict = af.AUTOPICK
    elif team_name == af.CRABBEHERBYTHEPUSSY[NAME_KEY]:
        team_dict = af.CRABBEHERBYTHEPUSSY
    elif team_name == af.EL_LADRON_DE_CABRAS[NAME_KEY]:
        team_dict = af.EL_LADRON_DE_CABRAS
    elif team_name == af.LALALAND[NAME_KEY]:
        team_dict = af.LALALAND
    elif team_name == af.MAGICS_JOHNSON[NAME_KEY]:
        team_dict = af.MAGICS_JOHNSON
    elif team_name == af.MCCURRY[NAME_KEY]:
        team_dict = af.MCCURRY
    elif team_name == af.NUNN_OF_YALL_BETTA[NAME_KEY]:
        team_dict = af.NUNN_OF_YALL_BETTA
    elif team_name == af.RUSTY_CUNTBROOKS[NAME_KEY]:
        team_dict = af.RUSTY_CUNTBROOKS
    elif team_name == af.SWAGGY_P[NAME_KEY]:
        team_dict = af.SWAGGY_P
    elif team_name == af.TVONS_TIP_TOP_TEAM[NAME_KEY]:
        team_dict = af.TVONS_TIP_TOP_TEAM
    elif team_name == af.WAKANDA_FOREVER[NAME_KEY]:
        team_dict = af.WAKANDA_FOREVER
    elif team_name == af.YOBITCH_TOPPIN_ME[NAME_KEY]:
        team_dict = af.YOBITCH_TOPPIN_ME
    return team_dict

### Get Stats Yahoo API

In [ ]:
league.player_stats([4660, 6048], req_type="lastweek")

In [ ]:
team1_name, team1_live_stats, team2_name, team2_live_stats = \
    get_matchup_teams_9cat_stat_dataframes(8, af.NUNN_OF_YALL_BETTA)

In [ ]:
team1_dict = get_team_dict_from_team_name(team1_name)
team1_dict

In [ ]:
name_id_dict, name_list = get_player_ids_names_in_fantasy_team(team1_dict)

In [ ]:
schedule_dataframe = get_game_information_in_season("2020")
week_start_date = "2021-02-14"
week_end_date = "2021-02-14"
games_left_dataframe, games_left_series = get_fantasy_week_games_dataframe(schedule_dataframe,
                                                                           week_start_date,
                                                                           week_end_date)
player_game_dictionary = get_player_games(games_left_series, name_list)

In [ ]:
# Nunn 9Cat averages
sc = af.yahoo_fantasy_api_authentication()
league = af.yahoo_fantasy_league(sc)
nunn_of_yall_betta_team = af.team_9cat_average_stats(team1_dict, league)

In [ ]:
predicted_stat_dataframe = get_predicted_player_weekly_9cat(player_game_dictionary,
                                                     nunn_of_yall_betta_team)
predicted_stat_dataframe

In [ ]:
datframe_with_total = get_total_row(predicted_stat_dataframe)
total_row_datafame, total_row_datafame_no_name = get_filtered_total_row(datframe_with_total, af
                                                                 .NUNN_OF_YALL_BETTA)
total_row_datafame

In [ ]:
total_row_datafame.reset_index(inplace=True, drop=True)
team1_live_stats = columns_to_dtype_float(team1_live_stats)
team1_live_stats.drop("PLAYER", axis=1, inplace=True)
total_row_datafame.drop("PLAYER", axis=1, inplace=True)

In [ ]:
team1_live_stats.add(total_row_datafame)

In [ ]:

a = pd.DataFrame()
a["PLAYER"] = "adasd"
b = pd.concat([a, team1_live_stats])
b["PLAYER"] = "dsf"
b


In [ ]:
def get_team_predicted_stats_dataframe(team_name, schedule_dataframe, week_start_date,
                                       week_end_date):
    """
    Function creates a dataframe containing the predicted 9cat stats for a teams roster dependent
    on the amount of games each player has left to play in the stated date range.
    @param team_name (string): name of Yahoo Fantasy team
    @param schedule_dataframe (pandas dataframe): dataframe containing the entire seasons
    NBA schedule
    @param week_start_date (string): start date of to begin schedule_dataframe filtering
    @param week_end_date (string): endd date of to begin schedule_dataframe filtering
    @return predicted_stats_dataframe: predicted 9cat stats for a teams roster
    @return team_dictionary (dictionary): dictionary of team from league
    """
    team_dictionary = get_team_dict_from_team_name(team_name)
    name_id_dictionary, name_list = get_player_ids_names_in_fantasy_team(team_dictionary)
    games_left_dataframe, games_left_series = get_fantasy_week_games_dataframe\
        (schedule_dataframe, week_start_date, week_end_date)
    player_game_dictionary = get_player_games(games_left_series, name_list)
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team_9cat_average_stats_dataframe = af.team_9cat_average_stats(team_dictionary, league)
    predicted_stats_dataframe = get_predicted_player_weekly_9cat(player_game_dictionary,
                                                                team_9cat_average_stats_dataframe)
    return predicted_stats_dataframe, team_dictionary

def get_clean_predicted_stats_dataframe(predicted_stats_dataframe, team_dict):
    """
    Function cleans the predicted stats dataframe by creating a total row and adding the name of
    the team via the team_dict parameter.
    @param predicted_stats_dataframe (pandas dataframe): predicted 9cat stats for a teams roster
    @param team_dict (dictionary): dictionary of team from league
    @return total_row_predicted_stats_datafame: 1x10 predicted 9cat dataframes
    @return total_row_predicted_stats_datafame_no_name: 1x9 predicted 9cat dataframes (without
    team name)
    """
    dataframe_with_total = get_total_row(predicted_stats_dataframe)
    total_row_predicted_stats_datafame, total_row_predicted_stats_datafame_no_name = \
        get_filtered_total_row(dataframe_with_total, team_dict)
    return total_row_predicted_stats_datafame, total_row_predicted_stats_datafame_no_name

def add_live_stats_and_predicted_dataframes(live_stats_dataframe,
                                            total_row_predicted_stats_datafame, team_name):
    """
    Function adds the dataframes containing live matchup stats and the predicted game stats. It
    returns a 1x10 dataframe with team name and 9cat stats.
    @param live_stats_dataframe (pandas dataframe): current matchup 9cat stats for a team roster
    @param total_row_predicted_stats_datafame (pandas dataframe): total row predicted 9cat stats
    for a team roster
    @param team_name (string): name of Yahoo Fantasy team
    @return live_plus_prediction_dataframe: 1X10 9cat live plus predicted 9cat stats dataframe
    """
    try:
        total_row_predicted_stats_datafame.reset_index(inplace=True, drop=True)
    except:
        pass
    live_stats_dataframe = columns_to_dtype_float(live_stats_dataframe)
    live_stats_dataframe.reset_index(inplace=True)
    try:
        total_row_predicted_stats_datafame.drop(PLAYER_COLUMN, axis=1, inplace=True)
    except:
        pass

    live_plus_prediction_dataframe = live_stats_dataframe.add(total_row_predicted_stats_datafame)
    print(live_plus_prediction_dataframe)
    temporary_dataframe = pd.DataFrame()
    temporary_dataframe[PLAYER_COLUMN] = team_name
    live_plus_prediction_dataframe = pd.concat([temporary_dataframe,
                                                live_plus_prediction_dataframe])
    live_plus_prediction_dataframe[PLAYER_COLUMN] = team_name
    return live_plus_prediction_dataframe

In [ ]:
total_row_predicted_stats_datafame

In [ ]:
team1_live_stats

In [ ]:
live_plus_prediction_dataframe = add_live_stats_and_predicted_dataframes(team1_live_stats,
                                            total_row_predicted_stats_datafame, team1_name)

live_plus_prediction_dataframe

In [ ]:
live_plus_prediction_dataframe.info()

In [ ]:
team1_name, team1_live_stats, team2_name, team2_live_stats, matchup_df = \
    get_matchup_teams_9cat_stat_dataframes(8, af.NUNN_OF_YALL_BETTA)

schedule_dataframe = get_game_information_in_season("2020")
week_start_date = "2021-02-14"
week_end_date = "2021-02-14"

In [ ]:
predicted_stats_dataframe, team_dictionary = get_team_predicted_stats_dataframe(team1_name,
                                                                                schedule_dataframe,
                                                             week_start_date, week_end_date)

total_row_predicted_stats_datafame, total_row_predicted_stats_datafame_no_name = \
    get_clean_predicted_stats_dataframe(predicted_stats_dataframe, team_dictionary)

live_plus_prediction_dataframe = add_live_stats_and_predicted_dataframes(team1_live_stats,
                                            total_row_predicted_stats_datafame, team1_name)

live_plus_prediction_dataframe

In [ ]:
def live_plus_prediction_matchup_dataframe(schedule_dataframe, team_name, current_date,
                                           week_end_date):
    """

    @param schedule_dataframe:
    @param team_name:
    @param current_date:
    @param week_end_date:
    @return:
    """
    predicted_stats_dataframe, team_dictionary = get_team_predicted_stats_dataframe\
        (team_name, schedule_dataframe, current_date, week_end_date)
    total_row_predicted_stats_datafame, total_row_predicted_stats_datafame_no_name = \
    get_clean_predicted_stats_dataframe(predicted_stats_dataframe, team_dictionary)
    live_plus_prediction_dataframe = add_live_stats_and_predicted_dataframes(team1_live_stats,
                                            total_row_predicted_stats_datafame, team_name)
    return live_plus_prediction_dataframe

In [ ]:
from datetime import date

schedule_dataframe = get_game_information_in_season("2020")

current_fantasy_week, week_start_date, week_end_date = get_week_current_week_information()

current_date = date.today()

team1_name, team1_live_stats, team2_name, team2_live_stats, matchup_stats_dataframe = \
    get_matchup_teams_9cat_stat_dataframes(current_fantasy_week, af.NUNN_OF_YALL_BETTA)

a = live_plus_prediction_matchup_dataframe(schedule_dataframe, team1_name, current_date,
                                           week_end_date)
a

In [ ]:
from datetime import date

def get_live_matchup_predictions(week, team_dict, team1_9cat_average_stats_data=None,
                             team2_9cat_average_stats_data=None, season="2020"):
    """

    @return:
    @rtype:
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team1_name, team1_live_stats, team2_name, team2_live_stats = \
        get_matchup_teams_9cat_stat_dataframes\
        (week, team_dict)
    team1_dict = get_team_dict_from_team_name(team1_name)
    team2_dict = get_team_dict_from_team_name(team2_name)
    name_id_dict, name_list = get_player_ids_names_in_fantasy_team(team_dict)
    schedule_dataframe = get_game_information_in_season(season)
    current_fantasy_week, week_start_date, week_end_date = get_week_current_week_information()
    week_start_date = date.today()
    games_left_dataframe, games_left_series = get_fantasy_week_games_dataframe(schedule_dataframe,
                                                                           week_start_date,
                                                                           week_end_date)
    player_game_dictionary = get_player_games(games_left_series, name_list)
    team1_9cat_average_stats_data = af.team_9cat_average_stats(team_dict, league)
    team1_predicted_stat_dataframe = get_predicted_player_weekly_9cat(player_game_dictionary,
                                                                team1_9cat_average_stats_data)
    team2_9cat_average_stats_data = af.team_9cat_average_stats(team_dict, league)
    team2_predicted_stat_dataframe = get_predicted_player_weekly_9cat(player_game_dictionary,
                                                                team2_9cat_average_stats_data)
    team1_dataframe_with_total = get_total_row(predicted_stat_dataframe)
    team1_total_row_datafame, team1_total_row_datafame_no_name = get_filtered_total_row\
        (team1_dataframe_with_total, team1_dict)
    team2_dataframe_with_total = get_total_row(predicted_stat_dataframe)
    team2_total_row_datafame, team2_total_row_datafame_no_name = get_filtered_total_row\
        (team2_dataframe_with_total, team2_dict)
    return team1_total_row_datafame ,team2_total_row_datafame

In [ ]:
week_start_date = date.today()

In [ ]:
week_start_date

In [ ]:
current_fantasy_week, week_start_date, week_end_date = get_week_current_week_information()

team1_name, team1_live_stats, team2_name, team2_live_stats, matchup_stats_dataframe = \
    get_matchup_teams_9cat_stat_dataframes(current_fantasy_week, af.NUNN_OF_YALL_BETTA)

In [ ]:
count_team1_win_list = 0
count_team2_win_list = 0
for column in matchup_stats_dataframe.columns[:-1]:
    if matchup_stats_dataframe[column][0] > matchup_stats_dataframe[column][1]:
        count_team1_win_list += 1
    elif matchup_stats_dataframe[column][0] < matchup_stats_dataframe[column][1]:
        count_team2_win_list += 1
    else:
        pass


if matchup_stats_dataframe["TOV"][0] > matchup_stats_dataframe["TOV"][1]:
    count_team2_win_list += 1
elif matchup_stats_dataframe["TOV"][0] < matchup_stats_dataframe["TOV"][1]:
    count_team1_win_list += 1
else:
    pass


In [ ]:
def move_column_inplace(df, col, pos):
    col = df.pop(col)
    df.insert(pos, col.name, col)

In [ ]:
team_stat_win_list = [count_team1_win_list, count_team2_win_list]
matchup_stats_dataframe["CATS"] = team_stat_win_list
move_column_inplace(matchup_stats_dataframe, "CATS", 0)

In [ ]:
matchup_stats_dataframe

In [ ]:
TURNOVERS_COLUMN = "TOV"
CATS_COLUMNS = "CATS"

def move_column_inplace(df, col, pos):
    col = df.pop(col)
    df.insert(pos, col.name, col)

def get_winning_cats(matchup_stats_dataframe):
    """
    Function counts, for each team (row) in dataframe, which team has better category values.
    This provides a score informing total categories that each team is winning.
    @param matchup_stats_dataframe:
    @return: dataframe with category win count column
    """
    count_team1_win_list = 0
    count_team2_win_list = 0
    for column in matchup_stats_dataframe.columns[:-1]:
        if matchup_stats_dataframe[column][0] > matchup_stats_dataframe[column][1]:
            count_team1_win_list += 1
        elif matchup_stats_dataframe[column][0] < matchup_stats_dataframe[column][1]:
            count_team2_win_list += 1
        else:
            pass
    if matchup_stats_dataframe[TURNOVERS_COLUMN][0] > matchup_stats_dataframe[TURNOVERS_COLUMN][1]:
        count_team2_win_list += 1
    elif matchup_stats_dataframe[TURNOVERS_COLUMN][0] < matchup_stats_dataframe[TURNOVERS_COLUMN][1]:
        count_team1_win_list += 1
    else:
        pass
    team_stat_win_list = [count_team1_win_list, count_team2_win_list]
    matchup_stats_dataframe[CATS_COLUMNS] = team_stat_win_list
    move_column_inplace(matchup_stats_dataframe, CATS_COLUMNS, 0)
    return matchup_stats_dataframe

def get_live_matchup_stats_pipeline(team_dictionary):
    """
    Function pipelines the process involved in returning, for a specified team, its matchups
    live stats and category win columns count
    @param team_dictionary: dictionary containing team information from league
    @return: dataframe with live matchup 9cat stats and category win count column
    """
    current_fantasy_week, week_start_date, week_end_date = get_week_current_week_information()
    team1_name, team1_live_stats, team2_name, team2_live_stats, matchup_stats_dataframe = \
        get_matchup_teams_9cat_stat_dataframes(current_fantasy_week, team_dictionary)
    matchup_stats_dataframe = get_winning_cats(matchup_stats_dataframe)
    return current_fantasy_week, team1_name, team1_live_stats, team2_name, team2_live_stats, \
           matchup_stats_dataframe

In [ ]:

get_live_matchup_stats_pipeline(af.NUNN_OF_YALL_BETTA)

In [ ]:
import requests
import pandas as pd
from src.data import all_functions as af

PLAYER_COLUMN = "PLAYER"
TEAM_KEY = "team_key"
PLAYER_ID_KEY = "player_id"
NAME_KEY = "name"
MEAN_ROW = "mean"
NAME_KEY = "name"
LSCD_KEY = "lscd"
MSCD_KEY = "mscd"
G_KEY = "g"
GID_KEY = "gid"
GAME_ID_KEY = "game_id"
GDATE_KEY = "gdte"
GAME_DATE_KEY= "game_date"
V_KEY = "v"
TID_KEY = "tid"
VISITORS_TEAM_ID_KEY = "visitors_tid"
VISITORS_KEY =  "visitors"
TC_KEY = "tc"
TN_KEY = "tn"
H_KEY = "h"
HOME_TEAM_ID_KEY = "home_tid"
HOME_KEY = "home"
TOTAL_ROW = "total"
INDEX_COLUMN = "index"
NBA_NAME_LA_CLIPPERS = "LA Clippers"
YAHOO_NAME_LA_CLIPPERS = "Los Angeles Clippers"
COLUMN_9CAT_DECIMAL_FORMAT = {af.FIELD_GOAL_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.FREE_THROW_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.THREES_MADE_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.POINTS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.REBOUNDS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.ASSITS_COLUMN : af.TWO_DECIMAL_FORMAT, af.STEALS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.BLOCKS_COLUMN : af.TWO_DECIMAL_FORMAT, af.TURNOVERS_COLUMN : af.TWO_DECIMAL_FORMAT}

def columns_to_dtype_float(dataframe):
    """
    Converts dataframe columns to dtype float.
    param dataframe (pandas dataframe): dataframe to change column types
    returns dataframe (pandas dataframe): dataframe with changed column types
    """
    for column in dataframe.columns:
        try:
            dataframe[column] = dataframe[column].astype(float)
        except ValueError:
            pass
    return dataframe

def get_and_format_team_9cat(team, league):
    """
    Function first finds the teams 9cat averages by calling the *team_9cat_average_stats*
    function. It then filters to find the team mean and cleans and formats this single row
    dataframe. It changes the dtype of the columns to be floats.
    param team (dictionary): team dictionary
    param league (yahoo oauth object): yahoo authentication object
    returns team_9cat (dataframe): dataframe of team average 9cats
    """
    team_9cat = af.team_9cat_average_stats(team, league)
    team_9cat = team_9cat.loc[MEAN_ROW]
    team_9cat[PLAYER_COLUMN] = team[NAME_KEY]
    team_9cat = pd.DataFrame(team_9cat).T
    team_9cat.drop(PLAYER_COLUMN, inplace=True, axis=1)
    team_9cat = columns_to_dtype_float(team_9cat)
    return team_9cat

def applymap_color_and_format(dataframe):
    """
    Function takes a dataframe containing 9cat stat columns and returns a color coded and decimal
    place formatted dataframe for visual ease.
    param dataframe (pandas dataframe): dataframe to format
    returns dataframe (pandas dataframe): dataframe with formatted and colored values
    """
    colored_matchup_difference = dataframe.style.applymap(
        af.color_negative_red, subset=pd.IndexSlice[:,[af.FIELD_GOAL_PERCENTAGE_COLUMN,
                                                       af.FREE_THROW_PERCENTAGE_COLUMN,
                                                       af.THREES_MADE_COLUMN, af.POINTS_COLUMN,
                                                       af.REBOUNDS_COLUMN, af.ASSITS_COLUMN,
                                                       af.STEALS_COLUMN, af.BLOCKS_COLUMN
                                                       ]]).applymap(
        af.color_negative_red_tov, subset=pd.IndexSlice[:,[af.TURNOVERS_COLUMN]]).format\
        (COLUMN_9CAT_DECIMAL_FORMAT)
    return colored_matchup_difference

def compare_h2h_team_9cat(team1, team2):
    """
    Function authenticates on Yahoo and creates creates two dataframes containing team roster
    9cat averages for each team specified in the parameters. It calculates the difference between
    both 9cat averages and finally it formats the values for visual ease.
    params team1 (dictionary): team dictionary
    params team2 (dictionary): team dictionary
    return matchup_difference (dataframe): team difference 9cat dataframe
    return colored_matchup_difference (dataframe): formatted team difference 9cat dataframe
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team1_9cat_stats = get_and_format_team_9cat(team1, league)
    team2_9cat_stats = get_and_format_team_9cat(team2, league)
    matchup_difference = team1_9cat_stats.sub(team2_9cat_stats)
    colored_matchup_difference = applymap_color_and_format(matchup_difference)
    return matchup_difference, colored_matchup_difference

def get_game_information_in_month(nba_json):
    """
    Function loops through a month (n) in nba_json and creates a dataframe containing game
    information, game_id, home/visitor team/team_id and game_date for each month.
    param nba_json (dictionary): dictionary containing game information
    returns game_dataframe (pandas dataframe): dataframe containing game information for a month
    """
    game_dataframe = pd.DataFrame()
    for n in range(len(nba_json)):
        game_dictionary = dict()
        game_dictionary[GID_KEY] = nba_json[n][GID_KEY]
        game_dictionary[GDATE_KEY] = nba_json[n][GDATE_KEY]
        game_dictionary[VISITORS_TEAM_ID_KEY] = nba_json[n][V_KEY][TID_KEY]
        game_dictionary[VISITORS_KEY] = nba_json[n][V_KEY][TC_KEY] + " " \
                                        + nba_json[n][V_KEY][TN_KEY]
        game_dictionary[HOME_TEAM_ID_KEY] = nba_json[n][H_KEY][TID_KEY]
        game_dictionary[HOME_KEY] = nba_json[n][H_KEY][TC_KEY] + " " \
                                    + nba_json[n][H_KEY][TN_KEY]
        game_dataframe = game_dataframe.append(game_dictionary, ignore_index=True)
    return game_dataframe

def get_game_information_in_season(season):
    """
    Function calls the NBA endpoint that returns the season schedule. It then loops through each
    month in that schedule to create a dataframe containing game information, game_id,
    home/visitor team/team_id and game_date for the season.
    param season (string): season for schedule
    returns season_games_dataframe (pandas dataframe): dataframe containing game information for
    a season
    """
    r = requests.get(f"http://data.nba"
                     f".com/data/10s/v2015/json/mobile_teams/nba/{season}/league/00_full_schedule"
                     f".json")
    game_information_json = r.json()[LSCD_KEY]
    season_games_dataframe = pd.DataFrame()
    for n in range(len(game_information_json)):
        month_n_game_dataframe = get_game_information_in_month\
            (game_information_json[n][MSCD_KEY][G_KEY])
        season_games_dataframe = season_games_dataframe.append(month_n_game_dataframe)
    return season_games_dataframe

def get_week_current_week_information():
    """
    Function finds the current Yahoo Fantasy week number and its start and end date.
    return current_fantasy_week (int): Yahoo Fantasy week number
    return week_start_date (string): start date of Yahoo Fantasy week number
    return week_end_date (string): end date of Yahoo Fantasy week number
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    current_fantasy_week = league.current_week()
    week_start_date = league.week_date_range(current_fantasy_week)[0]
    week_end_date = league.week_date_range(current_fantasy_week)[1]
    return current_fantasy_week, week_start_date, week_end_date

def get_next_week_information():
    """
    Function finds the next Yahoo Fantasy week number and its start and end date.
    return next_fantasy_week (int): Yahoo Fantasy week number
    return week_start_date (string): start date of Yahoo Fantasy week number
    return week_end_date (string): end date of Yahoo Fantasy week number
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    next_fantasy_week = league.current_week() + 1
    week_start_date = league.week_date_range(next_fantasy_week)[0]
    week_end_date = league.week_date_range(next_fantasy_week)[1]
    return next_fantasy_week, week_start_date, week_end_date

def get_player_ids_names_in_fantasy_team(team_dict):
    """
    Function finds the Yahoo Fantasy player ids and player names rostered by a team in the league
    param team_dict (dictionary): dictionary containing information about a owners team
    return player_id_name_team_list (list): Yahoo Fantasy player ids and names
    return player_name_list (list): Yahoo Fantasy player names
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team = league.to_team(team_dict[TEAM_KEY])
    team_roster = team.roster()
    player_id_name_team_list = [{PLAYER_ID_KEY : player[PLAYER_ID_KEY], NAME_KEY : player[NAME_KEY]}
                           for player in team_roster]
    player_name_list = [player[NAME_KEY] for player in player_id_name_team_list]
    return player_id_name_team_list, player_name_list

def get_fantasy_week_games_dataframe(season_schedule_dataframe, week_start_date, week_end_date):
    """
    Function filters the entire NBA season schedule on a start/end date to find the scheduled
    games between specific dates. It returns the amount of games each NBA team plays during that
    period of time.
    param season_schedule_dataframe (pandas dataframe): dataframe containing the entire seasons
    NBA schedule
    param week_start_date (string): date to begin filtering by
    param week_start_date (string): date to end filtering by
    return current_fantasy_games_week (pandas dataframe) dataframe containing the scheduled games
    in specified param dates
    return team_game_counts (pandas series): series containing count of games for each NBA team
    """
    filtered_games_week = season_schedule_dataframe[(season_schedule_dataframe[GDATE_KEY] >=
                                                   str(week_start_date)) &
                                                  (season_schedule_dataframe[GDATE_KEY] <=
                                                   str(week_end_date))]
    team_game_counts = filtered_games_week[HOME_KEY].append\
        (filtered_games_week[VISITORS_KEY]).value_counts()
    return filtered_games_week, team_game_counts

def get_player_games(team_game_counts_series, player_name_list):
    """
    Function creates a dictionary containing key/value pairs of player names/games in a series of
    NBA team game counts
    param team_game_counts_series (pandas series): series of team name and games playing
    param player_name_list (list): list of player names
    return player_games_dictionary (dictionary): key/value pairing of player names/games playing
    """
    player_games_dictionary = dict()
    team_game_counts_dataframe = pd.DataFrame(team_game_counts_series)
    team_game_counts_dataframe.reset_index(inplace=True)
    team_game_counts_dataframe = team_game_counts_dataframe.replace(NBA_NAME_LA_CLIPPERS,
                                                                    YAHOO_NAME_LA_CLIPPERS)
    for player in player_name_list:
        team = af.yahoo_player_team_and_jersey(player)[0]
        try:
            player_games_dictionary[player] = \
                team_game_counts_dataframe[team_game_counts_dataframe["index"] == team][0].iloc[0]
        except IndexError:
            player_games_dictionary[player] = 0
    return player_games_dictionary

def get_predicted_player_weekly_9cat(player_games_dictionary, team_9cat_average_stats):
    """
    Function is used to create a dataframe which contains predicted totals for a week of Yahoo
    Fantasy play. For each player, 9cat features are multiplied by the amount of games that
    player plays in one week.
    param player_games_dictionary (dictionary): key/value pairing of player names/games playing
    param team_9cat_average_stats (pandas dataframe): dataframe containing the season average
    9cat stats for each player on a team
    return predicted_9cat_stats (pandas dataframe): dataframe with 9cat averages multiplied by
    games played in a week
    """
    predicted_9cat_stats = pd.DataFrame()
    team_9cat_average_stats = columns_to_dtype_float(team_9cat_average_stats)
    for player in player_games_dictionary.keys():
        player_predicted_9cat_stats = pd.concat(
            [team_9cat_average_stats[team_9cat_average_stats[PLAYER_COLUMN] == player].iloc[:, :3],
            (team_9cat_average_stats[team_9cat_average_stats[PLAYER_COLUMN] == player].iloc[:, 3:]
            * player_games_dictionary[player])], axis=1)
        predicted_9cat_stats = player_predicted_9cat_stats.append(predicted_9cat_stats)
    predicted_9cat_stats.sort_index(inplace=True)
    return predicted_9cat_stats

def get_total_row(roster_dataframe):
    """
    Function creates a row in the provided parameter dataframe which contains the total for
    categories that should be summed and averages for categories that should be averaged
    param roster_dataframe (pandas dataframe): dataframe containing player week 9cat stats
    return roster_dataframe_with_total (pandas dataframe): dataframe containing total row
    """
    sum_9cat_column_dataframe = pd.DataFrame(roster_dataframe.iloc[:, 3:].sum()).T
    mean_9cat_column_dataframe =  pd.DataFrame(roster_dataframe.iloc[:, 1:3].mean()).T
    total_dataframe = pd.concat([mean_9cat_column_dataframe,  sum_9cat_column_dataframe], axis=1)
    total_dataframe[PLAYER_COLUMN] = TOTAL_ROW
    roster_dataframe_with_total = roster_dataframe.append(total_dataframe)
    roster_dataframe_with_total.reset_index(inplace=True, drop=True)
    return roster_dataframe_with_total

def get_teams_weekly_predicted_stats_pipeline(team_dict, week="next", season=2020,
                                              team_9cat_average_stats_data=None):
    """
    Function pipelines the process required to find a league teams weekly predicted 9cat totals.
    Initially, the number of games for each player in the provided parameter week is found.
    Average 9cat player stats are multiplied by that number and the entire roster stats are
    summed to provide a total. The dataframe is finally formatted.
    param team_dict (dictionary): dictionary of team from league
    param week (string): determines which week to calculate weekly games for
    param season (int): determines which season to get NBA schedule for
    param team_9cat_average_stats_data (pandas dataframe): dataframe containing player 9cat season
    averages
    return roster_dataframe_with_total (pandas dataframe): dataframe containing player 9cat
    predicted weekly stats and entire roster totals
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    if week == "next":
        fantasy_week, week_start_date, week_end_date = get_next_week_information()
    elif week == "current":
        fantasy_week, week_start_date, week_end_date = get_week_current_week_information()
    else:
        raise ValueError("Parameter 'week' must be 'current' or 'next'.")
    season_games_dataframe = get_game_information_in_season(season)
    filtered_games_week, team_game_counts = get_fantasy_week_games_dataframe\
        (season_games_dataframe, week_start_date, week_end_date)
    player_id_name_team_list, player_name_list = get_player_ids_names_in_fantasy_team(team_dict)
    player_games_dictionary = get_player_games(team_game_counts, player_name_list)
    if team_9cat_average_stats_data is None:
        team_9cat_average_stats_dataframe = af.team_9cat_average_stats(team_dict, league)
        predicted_weekly_9cat_stats = get_predicted_player_weekly_9cat(player_games_dictionary,
                                                                 team_9cat_average_stats_dataframe)
    else:
        predicted_weekly_9cat_stats = get_predicted_player_weekly_9cat(player_games_dictionary,
                                                                 team_9cat_average_stats_data)
    roster_dataframe_with_total = get_total_row(predicted_weekly_9cat_stats)
    roster_dataframe_with_total = af.format_dataframe_decimals(roster_dataframe_with_total)
    return roster_dataframe_with_total

def get_filtered_total_row(total_dataframe, team_dict):
    """
    Function filters a dataframe containing weekly total 9cat stats and returns the *total* row
    so that it can be used for differencing. The function appends the team name to the total row
    param total_dataframe (pandas dataframe): dataframe containing weekly total 9cat stats
    param team_dict (dictionary): dictionary of team from league
    return total_row_dataframe (pandas dataframe): single row dataframe containing total 9cat
    stats with team name
    return total_row_dataframe_no_name (pandas dataframe): single row dataframe containing total 9cat
    stats without team name
    """
    total_row_dataframe = total_dataframe[total_dataframe[PLAYER_COLUMN] == TOTAL_ROW]
    total_row_dataframe[PLAYER_COLUMN] = team_dict[NAME_KEY]
    total_row_dataframe.index = [TOTAL_ROW]
    total_row_dataframe_no_name = total_row_dataframe.drop(PLAYER_COLUMN, axis=1)
    return total_row_dataframe, total_row_dataframe_no_name

def weekly_matchup_evaluator(team1_dict, team2_dict, team1_9cat_average_stats_data=None,
                             team2_9cat_average_stats_data=None, week="next", season=2020):
    """

    """
    team1_roster_dataframe_with_total = get_teams_weekly_predicted_stats_pipeline(
        team1_dict, week=week, season=season,
        team_9cat_average_stats_data=team1_9cat_average_stats_data)
    team2_roster_dataframe_with_total = get_teams_weekly_predicted_stats_pipeline(
        team2_dict, week=week, season=season,
        team_9cat_average_stats_data=team2_9cat_average_stats_data)
    team1_total_row_dataframe, team1_total_row_dataframe_no_name = get_filtered_total_row\
        (team1_roster_dataframe_with_total, team1_dict)
    team2_total_row_dataframe, team2_total_row_dataframe_no_name = get_filtered_total_row\
        (team2_roster_dataframe_with_total, team2_dict)
    team1_total_row_dataframe_no_name = columns_to_dtype_float(team1_total_row_dataframe_no_name)
    team2_total_row_dataframe_no_name = columns_to_dtype_float(team2_total_row_dataframe_no_name)
    total_difference_dataframe = team1_total_row_dataframe_no_name.subtract\
        (team2_total_row_dataframe_no_name)
    total_difference_dataframe_colored = applymap_color_and_format(total_difference_dataframe)
    return total_difference_dataframe, total_difference_dataframe_colored

FANTASY_CONTENT_KEY = "fantasy_content"
LEAGUE_KEY = "league"
SCOREBOARD_KEY = "scoreboard"
ZERO_KEY = "0"
MATCHUPS_KEY = "matchups"
MATCHUP_KEY = "matchup"
TEAMS_KEY = "teams"
ONE_KEY = "1"
TEAM_KEY_2 = "team"
NAME_KEY = "name"
TEAM_STATS_KEY = "team_stats"
STATS_KEY = "stats"
STAT_KEY = "stat"
VALUE_KEY = "value"
MATCHUP_COLUMNS = [af.PLAYER_COLUMN] + af.NINE_CAT_COLUMNS

def get_teams_matchup_number(team_dict, matchups):
    """
    Function loops through returned the returned Yahoo Fantasy matchup nested dictionary to find
    the matchup number of the specified parameter team name.
    param team_dict (dictionary): dictionary containing information about a owners team
    param matchups (dictionary): league matchup information from specified week number
    return matchup (string): string value for the index of the specified teams nested matchup
    dictionary in the original matchup dictionary
    """
    for matchup in range(len(matchups[FANTASY_CONTENT_KEY][LEAGUE_KEY][1][SCOREBOARD_KEY][ZERO_KEY]\
                [MATCHUPS_KEY]) - 1):
        if matchups[FANTASY_CONTENT_KEY][LEAGUE_KEY][1][SCOREBOARD_KEY][ZERO_KEY]\
                [MATCHUPS_KEY][str(matchup)][MATCHUP_KEY][ZERO_KEY][TEAMS_KEY][ZERO_KEY]\
                [TEAM_KEY_2][0][2][NAME_KEY] == team_dict[NAME_KEY] \
                or matchups[FANTASY_CONTENT_KEY][LEAGUE_KEY][1][SCOREBOARD_KEY][ZERO_KEY]\
                [MATCHUPS_KEY][str(matchup)][MATCHUP_KEY][ZERO_KEY][TEAMS_KEY][ONE_KEY]\
                [TEAM_KEY_2][0][2][NAME_KEY] == team_dict[NAME_KEY]:
            return str(matchup)

def get_team_name_from_matchup(matchups, matchup_number, matchup_team_number):
    """
    Function returns the team name in a matchup.
    param matchups (dictionary): league matchup information from specified week number
    param matchup_number (string): matchup number for in matchups dictionary
    param matchup_team_number (string): either "0" or "1" indicating matchup team number
    return team_name (string): name of team in matchup
    """
    team_name = matchups[FANTASY_CONTENT_KEY][LEAGUE_KEY][1][SCOREBOARD_KEY][ZERO_KEY]\
                [MATCHUPS_KEY][matchup_number][MATCHUP_KEY][ZERO_KEY][TEAMS_KEY]\
                [matchup_team_number][TEAM_KEY_2][0][2][NAME_KEY]
    return team_name

def get_live_matchup_stats_dictionary(matchups, matchup_number, matchup_team_number):
    """
    Function returns a dictionary containing 9cat stats plus two more for a given matchup and
    matchup team number.
    param matchups (dictionary): league matchup information from specified week number
    param matchup_number (string): matchup number for in matchups dictionary
    param matchup_team_number (string): either "0" or "1" indicating matchup team number
    return live_matchup_stats (dictionary): dictionary of 9cat stats plus two more stats
    """
    live_matchup_stats = matchups[FANTASY_CONTENT_KEY][LEAGUE_KEY][1][SCOREBOARD_KEY][ZERO_KEY]\
                [MATCHUPS_KEY][matchup_number][MATCHUP_KEY][ZERO_KEY][TEAMS_KEY]\
                [matchup_team_number][TEAM_KEY_2][1][TEAM_STATS_KEY][STATS_KEY]
    return live_matchup_stats

def get_dataframe_from_live_matchup_stats(matchup_stats_dictionary, team_name):
    """
    Function creates a 1x10 dataframe containing the name of the team along with this live 9cat
    stats for a weeks matchup.
    param matchup_stats_dictionary (dictionary): dictionary containing the stats for a team
    param team_name (string): the name of the team
    return live_matchup_9cat_stats_dataframe (pandas dataframe): dataframe containing the name of the team along with this live 9cat
    stats for a weeks matchup
    """
    matchup_9cat_stats_list = list()
    for index in range(len(matchup_stats_dictionary)):
        if index != 0 and index != 2:
            matchup_9cat_stats_list.append(matchup_stats_dictionary[index][STAT_KEY][VALUE_KEY])
    values = [team_name] + matchup_9cat_stats_list
    live_matchup_9cat_stats_dataframe = pd.DataFrame(values).T
    live_matchup_9cat_stats_dataframe.columns = MATCHUP_COLUMNS
    live_matchup_9cat_stats_dataframe = live_matchup_9cat_stats_dataframe.replace("", 0)
    return live_matchup_9cat_stats_dataframe

# Format and find 9cat differences
def remove_player_and_float_convert(dataframe):
    """
    Function cleans dataframe by converting all columns to dtype float
    @param dataframe: dataframe with columns to convert
    @return: converted dataframe
    """
    for column in dataframe.columns:
        dataframe[column] = dataframe[column].astype(float)
    return dataframe

def format_3ptm_to_tov_as_int(dataframe):
    """
    Function changes the 3ptm to tov columns to dtype int needed for the live score dataframes.
    @param dataframe: dataframe with columns to convert
    @return: converted dataframe
    """
    for column in dataframe.columns[2:]:
        dataframe[column] = dataframe[column].astype(int)
    return dataframe

def get_matchup_teams_9cat_stat_dataframes(week, team_dict):
    """

    @param week:
    @type week:
    @param team_dict:
    @type team_dict:
    @return:
    @rtype:
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    matchups = league.matchups(week=week)
    matchup_number = get_teams_matchup_number(team_dict, matchups)
    first_team_name = get_team_name_from_matchup(matchups, matchup_number, "0")
    second_team_name = get_team_name_from_matchup(matchups, matchup_number, "1")
    first_team_stats_dictionary = get_live_matchup_stats_dictionary(matchups, matchup_number,
                                                                    "0")
    second_team_stats_dictionary = get_live_matchup_stats_dictionary(matchups, matchup_number,
                                                                     "1")
    first_team_stats_dataframe = get_dataframe_from_live_matchup_stats(first_team_stats_dictionary,
                                                              first_team_name)
    first_team_stats_dataframe = first_team_stats_dataframe.set_index(PLAYER_COLUMN)
    second_team_stats_dataframe = get_dataframe_from_live_matchup_stats(second_team_stats_dictionary,
                                                              second_team_name)
    second_team_stats_dataframe = second_team_stats_dataframe.set_index(PLAYER_COLUMN)
    matchup_stats_dataframe = first_team_stats_dataframe.append(second_team_stats_dataframe)
    matchup_stats_dataframe = remove_player_and_float_convert(matchup_stats_dataframe)
    matchup_stats_dataframe = format_3ptm_to_tov_as_int(matchup_stats_dataframe)
    return first_team_name, first_team_stats_dataframe, second_team_name, \
           second_team_stats_dataframe, matchup_stats_dataframe

def get_team_dict_from_team_name(team_name):
    """
    Function returns the team dictionary from a team name.
    @param team_name: name of fantasy team
    @type team_name: string
    @return: dictionary of fantasy team
    @rtype: dicitonary
    """
    if team_name == af.AUTOPICK[NAME_KEY]:
        team_dict = af.AUTOPICK
    elif team_name == af.CRABBEHERBYTHEPUSSY[NAME_KEY]:
        team_dict = af.CRABBEHERBYTHEPUSSY
    elif team_name == af.EL_LADRON_DE_CABRAS[NAME_KEY]:
        team_dict = af.EL_LADRON_DE_CABRAS
    elif team_name == af.LALALAND[NAME_KEY]:
        team_dict = af.LALALAND
    elif team_name == af.MAGICS_JOHNSON[NAME_KEY]:
        team_dict = af.MAGICS_JOHNSON
    elif team_name == af.MCCURRY[NAME_KEY]:
        team_dict = af.MCCURRY
    elif team_name == af.NUNN_OF_YALL_BETTA[NAME_KEY]:
        team_dict = af.NUNN_OF_YALL_BETTA
    elif team_name == af.RUSTY_CUNTBROOKS[NAME_KEY]:
        team_dict = af.RUSTY_CUNTBROOKS
    elif team_name == af.SWAGGY_P[NAME_KEY]:
        team_dict = af.SWAGGY_P
    elif team_name == af.TVONS_TIP_TOP_TEAM[NAME_KEY]:
        team_dict = af.TVONS_TIP_TOP_TEAM
    elif team_name == af.WAKANDA_FOREVER[NAME_KEY]:
        team_dict = af.WAKANDA_FOREVER
    elif team_name == af.YOBITCH_TOPPIN_ME[NAME_KEY]:
        team_dict = af.YOBITCH_TOPPIN_ME
    return team_dict

def get_team_predicted_stats_dataframe(team_name, schedule_dataframe, week_start_date,
                                       week_end_date):
    """
    Function creates a dataframe containing the predicted 9cat stats for a teams roster dependent
    on the amount of games each player has left to play in the stated date range.
    @param team_name (string): name of Yahoo Fantasy team
    @param schedule_dataframe (pandas dataframe): dataframe containing the entire seasons
    NBA schedule
    @param week_start_date (string): start date of to begin schedule_dataframe filtering
    @param week_end_date (string): endd date of to begin schedule_dataframe filtering
    @return predicted_stats_dataframe: predicted 9cat stats for a teams roster
    @return team_dictionary (dictionary): dictionary of team from league
    """
    team_dictionary = get_team_dict_from_team_name(team_name)
    name_id_dictionary, name_list = get_player_ids_names_in_fantasy_team(team_dictionary)
    games_left_dataframe, games_left_series = get_fantasy_week_games_dataframe\
        (schedule_dataframe, week_start_date, week_end_date)
    player_game_dictionary = get_player_games(games_left_series, name_list)
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team_9cat_average_stats_dataframe = af.team_9cat_average_stats(team_dictionary, league)
    predicted_stats_dataframe = get_predicted_player_weekly_9cat(player_game_dictionary,
                                                                team_9cat_average_stats_dataframe)
    return predicted_stats_dataframe, team_dictionary

def get_clean_predicted_stats_dataframe(predicted_stats_dataframe, team_dict):
    """
    Function cleans the predicted stats dataframe by creating a total row and adding the name of
    the team via the team_dict parameter.
    @param predicted_stats_dataframe (pandas dataframe): predicted 9cat stats for a teams roster
    @param team_dict (dictionary): dictionary of team from league
    @return total_row_predicted_stats_datafame: 1x10 predicted 9cat dataframes
    @return total_row_predicted_stats_datafame_no_name: 1x9 predicted 9cat dataframes (without
    team name)
    """
    dataframe_with_total = get_total_row(predicted_stats_dataframe)
    total_row_predicted_stats_datafame, total_row_predicted_stats_datafame_no_name = \
        get_filtered_total_row(dataframe_with_total, team_dict)
    return total_row_predicted_stats_datafame, total_row_predicted_stats_datafame_no_name

def add_live_stats_and_predicted_dataframes(live_stats_dataframe,
                                            total_row_predicted_stats_datafame, team_name):
    """
    Function adds the dataframes containing live matchup stats and the predicted game stats. It
    returns a 1x10 dataframe with team name and 9cat stats.
    @param live_stats_dataframe (pandas dataframe): current matchup 9cat stats for a team roster
    @param total_row_predicted_stats_datafame (pandas dataframe): total row predicted 9cat stats
    for a team roster
    @param team_name (string): name of Yahoo Fantasy team
    @return live_plus_prediction_dataframe: 1X10 9cat live plus predicted 9cat stats dataframe
    """
    total_row_predicted_stats_datafame.reset_index(inplace=True, drop=True)
    live_stats_dataframe = columns_to_dtype_float(live_stats_dataframe)
    total_row_predicted_stats_datafame.drop(PLAYER_COLUMN, axis=1, inplace=True)
    live_plus_prediction_dataframe = live_stats_dataframe.add(total_row_predicted_stats_datafame)
    temporary_dataframe = pd.DataFrame()
    temporary_dataframe[PLAYER_COLUMN] = team_name
    live_plus_prediction_dataframe = pd.concat([temporary_dataframe,
                                                live_plus_prediction_dataframe])
    live_plus_prediction_dataframe[PLAYER_COLUMN] = team_name
    return live_plus_prediction_dataframe

def live_plus_prediction_matchup_dataframe(schedule_dataframe, team_name, current_date,
                                           week_end_date):
    """

    @param schedule_dataframe:
    @param team_name:
    @param current_date:
    @param week_end_date:
    @return:
    """
    predicted_stats_dataframe, team_dictionary = get_team_predicted_stats_dataframe\
        (team_name, schedule_dataframe, current_date, week_end_date)
    total_row_predicted_stats_datafame, total_row_predicted_stats_datafame_no_name = \
    get_clean_predicted_stats_dataframe(predicted_stats_dataframe, team_dictionary)
    live_plus_prediction_dataframe = add_live_stats_and_predicted_dataframes(team1_live_stats,
                                            total_row_predicted_stats_datafame, team_name)
    return live_plus_prediction_dataframe

In [ ]:
first_team_name, first_team_stats_dataframe, second_team_name, \
           second_team_stats_dataframe, matchup_stats_dataframe = \
    get_matchup_teams_9cat_stat_dataframes(11, af.NUNN_OF_YALL_BETTA)

In [ ]:
print(first_team_name, second_team_name)

In [ ]:
league_standings_dictionary = league.standings()

In [ ]:
league_standings_dictionary

"".join([x["rank"] for x in league_standings_dictionary if x["name"] == first_team_name])

def get_teams_league_standing(team_name):
    """

    @param league_standing_dictionary:
    @param team_name:
    @return:
    """
    league_standings_dictionary = league.standings()
    standing = "".join([x["rank"] for x in league_standings_dictionary if x["name"] == team_name])
    return standing

In [ ]:
standing = get_teams_league_standing(league_standings_dictionary, "Russty Cuntbrooks")
standing

In [ ]:
get_next_week_information()[0]

In [ ]:
def format_team_standing_string(standing):
    """

    @param standing:
    @return:
    """
    if standing == "1":
        return standing + "st"
    elif standing == "2":
        return standing + "nd"
    else:
        return standing + "th"

In [ ]:
format_team_standing_string(standing)

In [ ]:
def get_team_names_from_matchup_week(week, team_dict):
    """
    Function returns both team names from a specified week and team name matchup
    @param week: week number
    @param team_dict: team information dictionary
    @return: names of both teams in matchup
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    matchups = league.matchups(week=week)
    matchup_number = get_teams_matchup_number(team_dict, matchups)
    first_team_name = get_team_name_from_matchup(matchups, matchup_number, "0")
    second_team_name = get_team_name_from_matchup(matchups, matchup_number, "1")
    return first_team_name, second_team_name

In [ ]:
get_team_names_from_matchup_week(11, af.NUNN_OF_YALL_BETTA)

In [ ]:

team_9cat_stats = get_team_roster_stats_dataframe(team_dict)